# Deep learning models for age prediction on EEG data

This notebook uses deep learning methods to predict the age of infants using EEG data. The EEG data is preprocessed as shown in the notebook 'Deep learning EEG_dataset preprocessing raw'. 

In [1]:
import sys, os, fnmatch, csv
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

sys.path.insert(0, os.path.dirname(os.getcwd()))

from config import PATH_DATA_PROCESSED_DL, PATH_MODELS

## Load preprocessed data

The data can be found in 'PATH_DATA_PROCESSED_DL'. This is a single folder with all the data and metadata. The EEG data is in the .zarr files and the metadata is in .csv files. The .zarr files are divided in chunks of 1-second epochs from the same subject and the metadata contains the information like the subject's identification code and age. 

Ideally, all the data (epochs + age as label) is loaded in memory and models are trained on this data. The data must be split in train/validation/test and subjects shouldn't be in more than one of these splits. However, the data set is too large to fit in memory.

Therefore, a generator is used that loads the data on the fly from the files, tackling the issue of limited memory. Additionally, because epochs often contain a lot of noise, multiple random epochs of the same subjects at the same age (e.g. 40) can be averaged to create a new epoch. 

In [2]:
%%time

# Load all the metadata

from sklearn.model_selection import train_test_split

# Step 1: Get all the files in the output folder
file_names = os.listdir(PATH_DATA_PROCESSED_DL)

# Step 2: Get the full paths of the files (without extensions)
files = [os.path.splitext(os.path.join(PATH_DATA_PROCESSED_DL, file_name))[0] for file_name in fnmatch.filter(file_names, "*.zarr")]

# Step 3: Load all the metadata
frames = []

for idx, feature_file in enumerate(files):
    df_metadata = pd.read_csv(feature_file.replace("processed_raw_", "processed_metadata_") + ".csv")
    frames.append(df_metadata)

df_metadata = pd.concat(frames) 

# Step 4: Add missing age information based on the age group the subject is in
df_metadata['age_months'].fillna(df_metadata['age_group'], inplace=True)
df_metadata['age_days'].fillna(df_metadata['age_group']*30, inplace=True)
df_metadata['age_years'].fillna(df_metadata['age_group']/12, inplace=True)

# Step 5: List all the unique subject IDs
subject_ids = sorted(list(set(df_metadata["code"].tolist())))

CPU times: user 6.25 s, sys: 1.05 s, total: 7.3 s
Wall time: 9.38 s


In [3]:
from sklearn.model_selection import train_test_split

IDs_train, IDs_temp = train_test_split(subject_ids, test_size=0.3, random_state=42)
IDs_test, IDs_val = train_test_split(IDs_temp, test_size=0.5, random_state=42)

In [4]:
df_metadata

,code,cnt_path,cnt_file,age_group,age_days,age_months,age_years
0,23,/Volumes/Seagate Expansion Drive/ePodium/Data/...,023_35_mc_mmn36,35,1052.0,35.066667,2.922222
0,337,/Volumes/Seagate Expansion Drive/ePodium/Data/...,337_23_jc_mmn_36_wk,23,692.0,23.066667,1.922222
0,456,/Volumes/Seagate Expansion Drive/ePodium/Data/...,456_23_md_mmn36_wk,23,691.0,23.033333,1.919444
0,328,/Volumes/Seagate Expansion Drive/ePodium/Data/...,328_23_jc_mmn36_wk,23,699.0,23.300000,1.941667
0,314,/Volumes/Seagate Expansion Drive/ePodium/Data/...,314_29_mmn_36_wk,29,877.0,29.233333,2.436111
...,...,...,...,...,...,...,...
0,348,/Volumes/Seagate Expansion Drive/ePodium/Data/...,348_29_jc_mmn25_wk,29,858.0,28.600000,2.383333
0,9,/Volumes/Seagate Expansion Drive/ePodium/Data/...,009_23_jc_mmn58,23,692.0,23.066667,1.922222
0,751,/Volumes/Seagate Expansion Drive/ePodium/Data/...,751-452-29m-jr-mmn36,29,869.0,28.966667,2.413889
0,348,/Volumes/Seagate Expansion Drive/ePodium/Data/...,348_17_jc_mmn25_wk,17,512.0,17.066667,1.422222


In [5]:
from dataset_generator import DataGenerator

In [6]:
train_generator_noise = DataGenerator(list_IDs = IDs_train,
                                      BASE_PATH = PATH_DATA_PROCESSED_DL,
                                      metadata = df_metadata,
                                      n_average = 30,
                                      batch_size = 10,
                                      gaussian_noise=0.01,
                                      iter_per_epoch = 30,
                                      n_timepoints = 501, 
                                      n_channels=30, 
                                      shuffle=True)

val_generator = DataGenerator(list_IDs = IDs_val,
                              BASE_PATH = PATH_DATA_PROCESSED_DL,
                              metadata = df_metadata,
                              n_average = 30,
                              batch_size = 10,
                              iter_per_epoch = 100,
                              n_timepoints = 501,
                              n_channels=30,
                              shuffle=True)

# Helper functions

In [7]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, Input, Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dropout, BatchNormalization, Dense, Conv1D, LeakyReLU, AveragePooling1D, Flatten, Reshape, MaxPooling1D
from tensorflow.keras.optimizers import Adam, Adadelta, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.metrics import RootMeanSquaredError, MeanAbsoluteError
import time

n_timesteps = 501
n_features = 30 
n_outputs = 1

input_shape = (n_timesteps, n_features)

In [8]:
def evaluate_model(model):
    """ Evaluates the model """
    model.evaluate(train_generator_noise)
    model.evaluate(val_generator)
    model.evaluate(test_generator) 
    
def print_few_predictions(model):
    """ Prints a few predictions, as a sanity check """
    x_test, y_test = test_generator.__getitem__(0)

    print(model.predict(x_test))
    print(y_test)
    
def plot_loss(history):
    """ Plots the MSE, RMSE, and MAE loss for the training and validation data over time """
    
    %matplotlib inline
    
    fig, (ax1, ax2, ax3) = plt.subplots(3, 1, sharex=True, figsize=(12,12), dpi=200)

    ax1.plot(history.history['loss'], label='training data')  
    min_loss = min(history.history['val_loss'])
    val_plot1 = ax1.plot(history.history['val_loss'], label='validation data')
    ax1.axhline(y = min_loss, color = val_plot1[0].get_color(), linestyle = '--') 
    x0,x1 = ax1.get_xlim()
    ax1.text(x1, min_loss, "{:.2f}".format(min_loss), ha='left', va='center')
    ax1.set_title('MSE loss')
    ax1.set_ylabel("MSE")
    ax1.set_xlabel("epochs")
    ax1.legend()

    ax2.plot(history.history['root_mean_squared_error'], label='training data')
    min_loss = min(history.history['val_root_mean_squared_error'])
    val_plot2 = ax2.plot(history.history['val_root_mean_squared_error'], label='validation data')
    ax2.axhline(y = min_loss, color = val_plot2[0].get_color(), linestyle = '--') 
    x0,x1 = ax2.get_xlim()
    ax2.text(x1, min_loss, "{:.2f}".format(min_loss), ha='left', va='center')
    ax2.set_title('RMSE loss')
    ax2.set_ylabel("RMSE")
    ax2.set_xlabel("epochs")
    ax2.legend()
    
    ax3.plot(history.history['mean_absolute_error'], label='training data')    
    min_loss = min(history.history['val_mean_absolute_error'])
    val_plot3 = ax3.plot(history.history['val_mean_absolute_error'], label='validation data')
    ax3.axhline(y = min_loss, color = val_plot3[0].get_color(), linestyle = '--') 
    x0,x1 = ax3.get_xlim()
    ax3.text(x1, min_loss, "{:.2f}".format(min_loss), ha='left', va='center')
    ax3.set_title('MAE loss')
    ax3.set_ylabel("MAE")
    ax3.set_xlabel("epochs")
    ax3.legend()

# Testing of different architectures

Below we will test multiple different architectures, most of them as discussed in "Deep learning for time series classification: a review", by Ismail Fawaz et al (2019). Most of them are inspired again on other papers. Refer to the Ismail Fawaz paper for the original papers.

1. Fully-connected NN
2. CNN
3. ResNet
4. Encoder
5. Time-CNN

Other architectures to test:

6. BLSTM-LSTM 
7. InceptionTime


# 1. Fully connected NN

In [59]:
def fully_connected_model():
    """ Returns the fully connected model from Ismail Fawaz et al. (2019). """

    input_layer = keras.layers.Input(input_shape)

    input_layer_flattened = keras.layers.Flatten()(input_layer)

    layer_1 = keras.layers.Dropout(0.1)(input_layer_flattened)
    layer_1 = keras.layers.Dense(500, activation='relu')(layer_1)

    layer_2 = keras.layers.Dropout(0.2)(layer_1)
    layer_2 = keras.layers.Dense(500, activation='relu')(layer_2)

    layer_3 = keras.layers.Dropout(0.2)(layer_2)
    layer_3 = keras.layers.Dense(500, activation='relu')(layer_3)

    output_layer = keras.layers.Dropout(0.3)(layer_3)
    output_layer = keras.layers.Dense(1)(output_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model

In [60]:
model = fully_connected_model()

optimizer = Adadelta(learning_rate=0.01)    
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

# 01 seems to be incorrect (makes too many predictions, changed model)
# Fully_connected_regressor_01: MSE, Adadelta, N_average=30, 5000 epochs, ES=1000, RLR=200, gaussian=0.01
# Fully_connected_regressor_02: MSE, Adadelta, N_average=30, 5000 epochs, ES=1000, RLR=200, gaussian=0.01
output_filename = 'Fully_connected_regressor_FINAL_LOCAL'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=1200, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=200, min_lr=0.0001, verbose=1)

In [61]:
%%time

epochs = 5000

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs, 
                    callbacks=[checkpointer, earlystopper, reduce_lr])

Epoch 1/5000
147/147 [==============================] - 118s 795ms/step - loss: 682.6837 - root_mean_squared_error: 26.1022 - mean_absolute_error: 23.8852 - val_loss: 255.4768 - val_root_mean_squared_error: 15.9836 - val_mean_absolute_error: 13.1440

Epoch 00001: val_loss improved from inf to 255.47679, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Fully_connected_regressor_FINAL_LOCAL.hdf5
Epoch 2/5000
147/147 [==============================] - 94s 641ms/step - loss: 228.3679 - root_mean_squared_error: 15.1027 - mean_absolute_error: 12.1350 - val_loss: 209.9330 - val_root_mean_squared_error: 14.4891 - val_mean_absolute_error: 11.7058

Epoch 00002: val_loss improved from 255.47679 to 209.93295, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Fully_connected_regressor_FINAL_LOCAL.hdf5
Epoch 3/5000
147/147 [==============================] - 100s 680ms/step - loss: 215.7140 - root_mean_squared_error: 1


Epoch 00022: val_loss did not improve from 91.48636
Epoch 23/5000
147/147 [==============================] - 65s 441ms/step - loss: 91.9378 - root_mean_squared_error: 9.5847 - mean_absolute_error: 7.6501 - val_loss: 97.8368 - val_root_mean_squared_error: 9.8913 - val_mean_absolute_error: 7.6850

Epoch 00023: val_loss did not improve from 91.48636
Epoch 24/5000
147/147 [==============================] - 66s 447ms/step - loss: 97.2309 - root_mean_squared_error: 9.8594 - mean_absolute_error: 7.7814 - val_loss: 96.0407 - val_root_mean_squared_error: 9.8000 - val_mean_absolute_error: 7.6057

Epoch 00024: val_loss did not improve from 91.48636
Epoch 25/5000
147/147 [==============================] - 65s 439ms/step - loss: 93.5810 - root_mean_squared_error: 9.6709 - mean_absolute_error: 7.7350 - val_loss: 94.8326 - val_root_mean_squared_error: 9.7382 - val_mean_absolute_error: 7.8229

Epoch 00025: val_loss did not improve from 91.48636
Epoch 26/5000
147/147 [==============================] -


Epoch 00075: val_loss did not improve from 72.18785
Epoch 76/5000
147/147 [==============================] - 128s 873ms/step - loss: 69.3676 - root_mean_squared_error: 8.3256 - mean_absolute_error: 6.6992 - val_loss: 83.0663 - val_root_mean_squared_error: 9.1141 - val_mean_absolute_error: 7.1516

Epoch 00076: val_loss did not improve from 72.18785
Epoch 77/5000
147/147 [==============================] - 100s 680ms/step - loss: 75.7138 - root_mean_squared_error: 8.6953 - mean_absolute_error: 7.0309 - val_loss: 73.3478 - val_root_mean_squared_error: 8.5643 - val_mean_absolute_error: 6.6957

Epoch 00077: val_loss did not improve from 72.18785
Epoch 78/5000
147/147 [==============================] - 79s 539ms/step - loss: 74.4720 - root_mean_squared_error: 8.6258 - mean_absolute_error: 6.7729 - val_loss: 81.7555 - val_root_mean_squared_error: 9.0419 - val_mean_absolute_error: 7.3667

Epoch 00078: val_loss did not improve from 72.18785
Epoch 79/5000
147/147 [==============================]

147/147 [==============================] - 80s 544ms/step - loss: 71.4383 - root_mean_squared_error: 8.4498 - mean_absolute_error: 6.7421 - val_loss: 78.7933 - val_root_mean_squared_error: 8.8766 - val_mean_absolute_error: 6.9423

Epoch 00103: val_loss did not improve from 68.35943
Epoch 104/5000
147/147 [==============================] - 77s 526ms/step - loss: 70.7396 - root_mean_squared_error: 8.4061 - mean_absolute_error: 6.6956 - val_loss: 85.7973 - val_root_mean_squared_error: 9.2627 - val_mean_absolute_error: 7.3996

Epoch 00104: val_loss did not improve from 68.35943
Epoch 105/5000
147/147 [==============================] - 83s 564ms/step - loss: 71.3263 - root_mean_squared_error: 8.4390 - mean_absolute_error: 6.8572 - val_loss: 91.9392 - val_root_mean_squared_error: 9.5885 - val_mean_absolute_error: 7.5725

Epoch 00105: val_loss did not improve from 68.35943
Epoch 106/5000
147/147 [==============================] - 83s 564ms/step - loss: 63.0632 - root_mean_squared_error: 7.935


Epoch 00156: val_loss did not improve from 62.93748
Epoch 157/5000
147/147 [==============================] - 123s 840ms/step - loss: 61.7213 - root_mean_squared_error: 7.8541 - mean_absolute_error: 6.3251 - val_loss: 74.8005 - val_root_mean_squared_error: 8.6487 - val_mean_absolute_error: 6.7691

Epoch 00157: val_loss did not improve from 62.93748
Epoch 158/5000
147/147 [==============================] - 123s 835ms/step - loss: 70.5625 - root_mean_squared_error: 8.3916 - mean_absolute_error: 6.7807 - val_loss: 67.0195 - val_root_mean_squared_error: 8.1865 - val_mean_absolute_error: 6.4486

Epoch 00158: val_loss did not improve from 62.93748
Epoch 159/5000
147/147 [==============================] - 123s 835ms/step - loss: 65.6643 - root_mean_squared_error: 8.1008 - mean_absolute_error: 6.5499 - val_loss: 83.6006 - val_root_mean_squared_error: 9.1433 - val_mean_absolute_error: 7.2646

Epoch 00159: val_loss did not improve from 62.93748
Epoch 160/5000
147/147 [==========================

...

147/147 [==============================] - 66s 446ms/step - loss: 41.7783 - root_mean_squared_error: 6.4577 - mean_absolute_error: 5.1011 - val_loss: 64.9667 - val_root_mean_squared_error: 8.0602 - val_mean_absolute_error: 6.1742

Epoch 01546: val_loss did not improve from 44.73439
Epoch 1547/5000
147/147 [==============================] - 65s 445ms/step - loss: 44.2982 - root_mean_squared_error: 6.6542 - mean_absolute_error: 5.3782 - val_loss: 69.4343 - val_root_mean_squared_error: 8.3327 - val_mean_absolute_error: 6.4474

Epoch 01547: val_loss did not improve from 44.73439
Epoch 1548/5000
147/147 [==============================] - 65s 444ms/step - loss: 48.5676 - root_mean_squared_error: 6.9570 - mean_absolute_error: 5.3176 - val_loss: 57.5839 - val_root_mean_squared_error: 7.5884 - val_mean_absolute_error: 5.7849

Epoch 01548: val_loss did not improve from 44.73439
Epoch 1549/5000
147/147 [==============================] - 66s 447ms/step - loss: 47.2302 - root_mean_squared_error: 6.

147/147 [==============================] - 67s 454ms/step - loss: 41.8431 - root_mean_squared_error: 6.4639 - mean_absolute_error: 5.1290 - val_loss: 58.9766 - val_root_mean_squared_error: 7.6796 - val_mean_absolute_error: 6.0138

Epoch 01600: val_loss did not improve from 44.73439
Epoch 1601/5000
147/147 [==============================] - 66s 452ms/step - loss: 43.8649 - root_mean_squared_error: 6.6212 - mean_absolute_error: 5.2665 - val_loss: 69.1965 - val_root_mean_squared_error: 8.3184 - val_mean_absolute_error: 6.4007

Epoch 01601: val_loss did not improve from 44.73439
Epoch 1602/5000
147/147 [==============================] - 66s 452ms/step - loss: 44.4146 - root_mean_squared_error: 6.6622 - mean_absolute_error: 5.3233 - val_loss: 72.7551 - val_root_mean_squared_error: 8.5297 - val_mean_absolute_error: 6.4946

Epoch 01602: val_loss did not improve from 44.73439
Epoch 1603/5000
147/147 [==============================] - 67s 454ms/step - loss: 47.9141 - root_mean_squared_error: 6.

147/147 [==============================] - 67s 454ms/step - loss: 47.4647 - root_mean_squared_error: 6.8855 - mean_absolute_error: 5.4303 - val_loss: 60.2468 - val_root_mean_squared_error: 7.7619 - val_mean_absolute_error: 5.8359

Epoch 01654: val_loss did not improve from 44.73439
Epoch 1655/5000
147/147 [==============================] - 66s 451ms/step - loss: 43.5075 - root_mean_squared_error: 6.5927 - mean_absolute_error: 5.2681 - val_loss: 57.0974 - val_root_mean_squared_error: 7.5563 - val_mean_absolute_error: 5.8232

Epoch 01655: val_loss did not improve from 44.73439
Epoch 1656/5000
147/147 [==============================] - 66s 452ms/step - loss: 43.4191 - root_mean_squared_error: 6.5859 - mean_absolute_error: 5.2003 - val_loss: 68.8128 - val_root_mean_squared_error: 8.2953 - val_mean_absolute_error: 6.4625

Epoch 01656: val_loss did not improve from 44.73439
Epoch 1657/5000
147/147 [==============================] - 66s 451ms/step - loss: 44.4076 - root_mean_squared_error: 6.

147/147 [==============================] - 67s 453ms/step - loss: 44.7354 - root_mean_squared_error: 6.6844 - mean_absolute_error: 5.2025 - val_loss: 68.2076 - val_root_mean_squared_error: 8.2588 - val_mean_absolute_error: 6.2830

Epoch 01708: val_loss did not improve from 44.73439
Epoch 1709/5000
147/147 [==============================] - 66s 452ms/step - loss: 42.0178 - root_mean_squared_error: 6.4805 - mean_absolute_error: 5.2194 - val_loss: 73.0630 - val_root_mean_squared_error: 8.5477 - val_mean_absolute_error: 6.5796

Epoch 01709: val_loss did not improve from 44.73439
Epoch 1710/5000
147/147 [==============================] - 66s 450ms/step - loss: 43.8084 - root_mean_squared_error: 6.6175 - mean_absolute_error: 5.2033 - val_loss: 65.8975 - val_root_mean_squared_error: 8.1177 - val_mean_absolute_error: 6.0789

Epoch 01710: val_loss did not improve from 44.73439
Epoch 1711/5000
147/147 [==============================] - 66s 450ms/step - loss: 47.9726 - root_mean_squared_error: 6.

147/147 [==============================] - 75s 511ms/step - loss: 47.9583 - root_mean_squared_error: 6.9213 - mean_absolute_error: 5.4765 - val_loss: 60.9497 - val_root_mean_squared_error: 7.8070 - val_mean_absolute_error: 5.8770

Epoch 01762: val_loss did not improve from 44.73439
Epoch 1763/5000
147/147 [==============================] - 74s 504ms/step - loss: 42.5086 - root_mean_squared_error: 6.5139 - mean_absolute_error: 5.0036 - val_loss: 50.8092 - val_root_mean_squared_error: 7.1281 - val_mean_absolute_error: 5.4471

Epoch 01763: val_loss did not improve from 44.73439
Epoch 1764/5000
147/147 [==============================] - 72s 488ms/step - loss: 43.9358 - root_mean_squared_error: 6.6269 - mean_absolute_error: 5.2571 - val_loss: 72.8457 - val_root_mean_squared_error: 8.5350 - val_mean_absolute_error: 6.5517

Epoch 01764: val_loss did not improve from 44.73439
Epoch 1765/5000
147/147 [==============================] - 74s 502ms/step - loss: 45.5939 - root_mean_squared_error: 6.

# 2. CNN

In [65]:
def cnn_model():
    """ Returns the CNN (FCN) model from Ismail Fawaz et al. (2019). """

    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=128, kernel_size=8, padding='same')(input_layer)
    conv1 = keras.layers.BatchNormalization()(conv1)
    conv1 = keras.layers.Activation(activation='relu')(conv1)

    conv2 = keras.layers.Conv1D(filters=256, kernel_size=5, padding='same')(conv1)
    conv2 = keras.layers.BatchNormalization()(conv2)
    conv2 = keras.layers.Activation('relu')(conv2)

    conv3 = keras.layers.Conv1D(128, kernel_size=3, padding='same')(conv2)
    conv3 = keras.layers.BatchNormalization()(conv3)
    conv3 = keras.layers.Activation('relu')(conv3)

    pooling_layer = tf.keras.layers.AveragePooling1D(pool_size=50)(conv3)

    flat_layer = tf.keras.layers.Flatten()(pooling_layer)
    dense_layer = tf.keras.layers.Dense(128, activation='relu')(flat_layer)

    output_layer = tf.keras.layers.Dense(1)(dense_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model 


# def cnn_model():
#     """ Returns the CNN (FCN) model from Ismail Fawaz et al. (2019). """

#     input_layer = keras.layers.Input(input_shape)

#     conv1 = keras.layers.Conv1D(filters=128, kernel_size=8, padding='same')(input_layer)
#     conv1 = keras.layers.BatchNormalization()(conv1)
#     conv1 = keras.layers.Activation(activation='relu')(conv1)

#     conv2 = keras.layers.Conv1D(filters=256, kernel_size=5, padding='same')(conv1)
#     conv2 = keras.layers.BatchNormalization()(conv2)
#     conv2 = keras.layers.Activation('relu')(conv2)

#     conv3 = keras.layers.Conv1D(128, kernel_size=3, padding='same')(conv2)
#     conv3 = keras.layers.BatchNormalization()(conv3)
#     conv3 = keras.layers.Activation('relu')(conv3)

#     gap_layer = keras.layers.GlobalAveragePooling1D()(conv3)

#     output_layer = keras.layers.Dense(1)(gap_layer)

#     model = keras.models.Model(inputs=input_layer, outputs=output_layer)

#     return model 

In [66]:
model = cnn_model()

optimizer = Adam(learning_rate=0.01)    
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

# CNN_regressor_01: MSE, Adam, N_average=30, 2000 epochs, ES=250, RLR=50, gaussian=0.01
output_filename = 'CNN_regressor_FINAL_LOCAL'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=300, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=0.0001, verbose=1)

In [67]:
%%time

epochs = 2000

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs, 
                    callbacks=[checkpointer, earlystopper, reduce_lr])

Epoch 1/2000
147/147 [==============================] - 79s 518ms/step - loss: 190.9170 - root_mean_squared_error: 13.5743 - mean_absolute_error: 10.9382 - val_loss: 198.6267 - val_root_mean_squared_error: 14.0935 - val_mean_absolute_error: 10.9390

Epoch 00001: val_loss improved from inf to 198.62669, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/CNN_regressor_FINAL_LOCAL.hdf5
Epoch 2/2000
147/147 [==============================] - 76s 516ms/step - loss: 107.6590 - root_mean_squared_error: 10.3642 - mean_absolute_error: 8.4756 - val_loss: 125.5282 - val_root_mean_squared_error: 11.2039 - val_mean_absolute_error: 8.9647

Epoch 00002: val_loss improved from 198.62669 to 125.52819, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/CNN_regressor_FINAL_LOCAL.hdf5
Epoch 3/2000
147/147 [==============================] - 79s 535ms/step - loss: 108.4668 - root_mean_squared_error: 10.4025 - mean_absolute_error

147/147 [==============================] - 71s 484ms/step - loss: 75.8999 - root_mean_squared_error: 8.7101 - mean_absolute_error: 7.0811 - val_loss: 85.6599 - val_root_mean_squared_error: 9.2553 - val_mean_absolute_error: 7.2914

Epoch 00026: val_loss did not improve from 63.93641
Epoch 27/2000
147/147 [==============================] - 71s 486ms/step - loss: 74.5039 - root_mean_squared_error: 8.6309 - mean_absolute_error: 7.0652 - val_loss: 61.0824 - val_root_mean_squared_error: 7.8155 - val_mean_absolute_error: 6.4959

Epoch 00027: val_loss improved from 63.93641 to 61.08244, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/CNN_regressor_FINAL_LOCAL.hdf5
Epoch 28/2000
147/147 [==============================] - 72s 490ms/step - loss: 69.2544 - root_mean_squared_error: 8.3211 - mean_absolute_error: 6.8006 - val_loss: 84.9145 - val_root_mean_squared_error: 9.2149 - val_mean_absolute_error: 7.4013

Epoch 00028: val_loss did not improve from 61.0


Epoch 00052: val_loss did not improve from 56.05204
Epoch 53/2000
147/147 [==============================] - 72s 488ms/step - loss: 67.2519 - root_mean_squared_error: 8.1882 - mean_absolute_error: 6.5873 - val_loss: 64.9370 - val_root_mean_squared_error: 8.0583 - val_mean_absolute_error: 6.3374

Epoch 00053: val_loss did not improve from 56.05204
Epoch 54/2000
147/147 [==============================] - 71s 485ms/step - loss: 63.9743 - root_mean_squared_error: 7.9936 - mean_absolute_error: 6.5278 - val_loss: 67.0580 - val_root_mean_squared_error: 8.1889 - val_mean_absolute_error: 6.5426

Epoch 00054: val_loss did not improve from 56.05204
Epoch 55/2000
147/147 [==============================] - 72s 489ms/step - loss: 68.6062 - root_mean_squared_error: 8.2801 - mean_absolute_error: 6.6880 - val_loss: 58.7575 - val_root_mean_squared_error: 7.6653 - val_mean_absolute_error: 5.9437

Epoch 00055: val_loss did not improve from 56.05204
Epoch 56/2000
147/147 [==============================] -

147/147 [==============================] - 70s 475ms/step - loss: 49.6545 - root_mean_squared_error: 7.0451 - mean_absolute_error: 5.6005 - val_loss: 63.5443 - val_root_mean_squared_error: 7.9715 - val_mean_absolute_error: 6.1563

Epoch 00107: val_loss did not improve from 56.05204
Epoch 108/2000
147/147 [==============================] - 70s 475ms/step - loss: 57.6526 - root_mean_squared_error: 7.5874 - mean_absolute_error: 6.0976 - val_loss: 60.5594 - val_root_mean_squared_error: 7.7820 - val_mean_absolute_error: 5.9640

Epoch 00108: val_loss did not improve from 56.05204
Epoch 109/2000
147/147 [==============================] - 69s 473ms/step - loss: 57.8630 - root_mean_squared_error: 7.6052 - mean_absolute_error: 6.1646 - val_loss: 61.6746 - val_root_mean_squared_error: 7.8533 - val_mean_absolute_error: 5.9814

Epoch 00109: val_loss did not improve from 56.05204
Epoch 110/2000
147/147 [==============================] - 71s 484ms/step - loss: 55.4931 - root_mean_squared_error: 7.444


Epoch 00160: val_loss did not improve from 50.36514
Epoch 161/2000
147/147 [==============================] - 69s 471ms/step - loss: 50.1563 - root_mean_squared_error: 7.0790 - mean_absolute_error: 5.5786 - val_loss: 59.4648 - val_root_mean_squared_error: 7.7113 - val_mean_absolute_error: 5.6146

Epoch 00161: val_loss did not improve from 50.36514
Epoch 162/2000
147/147 [==============================] - 70s 474ms/step - loss: 49.0528 - root_mean_squared_error: 6.9904 - mean_absolute_error: 5.4613 - val_loss: 63.9703 - val_root_mean_squared_error: 7.9981 - val_mean_absolute_error: 6.5141

Epoch 00162: val_loss did not improve from 50.36514
Epoch 163/2000
147/147 [==============================] - 69s 473ms/step - loss: 51.6989 - root_mean_squared_error: 7.1844 - mean_absolute_error: 5.8262 - val_loss: 48.8641 - val_root_mean_squared_error: 6.9903 - val_mean_absolute_error: 5.4851

Epoch 00163: val_loss improved from 50.36514 to 48.86412, saving model to /Users/bbruns/Desktop/Studie/Th

...


Epoch 00433: val_loss did not improve from 41.45988
Epoch 434/2000
147/147 [==============================] - 71s 483ms/step - loss: 40.5593 - root_mean_squared_error: 6.3646 - mean_absolute_error: 4.9123 - val_loss: 50.3381 - val_root_mean_squared_error: 7.0949 - val_mean_absolute_error: 5.4572

Epoch 00434: val_loss did not improve from 41.45988
Epoch 435/2000
147/147 [==============================] - 71s 484ms/step - loss: 44.7727 - root_mean_squared_error: 6.6884 - mean_absolute_error: 5.3363 - val_loss: 54.9592 - val_root_mean_squared_error: 7.4134 - val_mean_absolute_error: 5.6388

Epoch 00435: val_loss did not improve from 41.45988
Epoch 436/2000
147/147 [==============================] - 70s 479ms/step - loss: 44.9216 - root_mean_squared_error: 6.7007 - mean_absolute_error: 5.3204 - val_loss: 59.2364 - val_root_mean_squared_error: 7.6965 - val_mean_absolute_error: 5.9058

Epoch 00436: val_loss did not improve from 41.45988

Epoch 00436: ReduceLROnPlateau reducing learning rat

147/147 [==============================] - 71s 485ms/step - loss: 42.3380 - root_mean_squared_error: 6.5038 - mean_absolute_error: 5.1885 - val_loss: 59.3085 - val_root_mean_squared_error: 7.7012 - val_mean_absolute_error: 5.8898

Epoch 00461: val_loss did not improve from 41.45988
Epoch 462/2000
147/147 [==============================] - 71s 482ms/step - loss: 41.4111 - root_mean_squared_error: 6.4333 - mean_absolute_error: 5.0963 - val_loss: 62.7172 - val_root_mean_squared_error: 7.9194 - val_mean_absolute_error: 5.9619

Epoch 00462: val_loss did not improve from 41.45988
Epoch 463/2000
147/147 [==============================] - 71s 482ms/step - loss: 43.4433 - root_mean_squared_error: 6.5902 - mean_absolute_error: 5.2540 - val_loss: 61.6186 - val_root_mean_squared_error: 7.8498 - val_mean_absolute_error: 6.1113

Epoch 00463: val_loss did not improve from 41.45988
Epoch 464/2000
147/147 [==============================] - 71s 483ms/step - loss: 40.6894 - root_mean_squared_error: 6.377


Epoch 00515: val_loss did not improve from 41.45988
Epoch 516/2000
147/147 [==============================] - 68s 466ms/step - loss: 44.7734 - root_mean_squared_error: 6.6859 - mean_absolute_error: 5.2820 - val_loss: 58.1239 - val_root_mean_squared_error: 7.6239 - val_mean_absolute_error: 5.8947

Epoch 00516: val_loss did not improve from 41.45988
Epoch 517/2000
147/147 [==============================] - 68s 466ms/step - loss: 41.2478 - root_mean_squared_error: 6.4159 - mean_absolute_error: 5.0405 - val_loss: 60.0328 - val_root_mean_squared_error: 7.7481 - val_mean_absolute_error: 6.0178

Epoch 00517: val_loss did not improve from 41.45988
Epoch 518/2000
147/147 [==============================] - 69s 468ms/step - loss: 45.1793 - root_mean_squared_error: 6.7205 - mean_absolute_error: 5.2910 - val_loss: 54.8106 - val_root_mean_squared_error: 7.4034 - val_mean_absolute_error: 5.8186

Epoch 00518: val_loss did not improve from 41.45988
Epoch 519/2000
147/147 [=============================


Epoch 00570: val_loss did not improve from 41.45988
Epoch 571/2000
147/147 [==============================] - 69s 468ms/step - loss: 43.1035 - root_mean_squared_error: 6.5612 - mean_absolute_error: 5.2270 - val_loss: 59.5143 - val_root_mean_squared_error: 7.7146 - val_mean_absolute_error: 5.9479

Epoch 00571: val_loss did not improve from 41.45988
Epoch 572/2000
147/147 [==============================] - 69s 470ms/step - loss: 41.5968 - root_mean_squared_error: 6.4477 - mean_absolute_error: 5.1586 - val_loss: 64.7121 - val_root_mean_squared_error: 8.0444 - val_mean_absolute_error: 6.1822

Epoch 00572: val_loss did not improve from 41.45988
Epoch 573/2000
147/147 [==============================] - 69s 468ms/step - loss: 44.5047 - root_mean_squared_error: 6.6697 - mean_absolute_error: 5.2321 - val_loss: 57.3097 - val_root_mean_squared_error: 7.5703 - val_mean_absolute_error: 5.7703

Epoch 00573: val_loss did not improve from 41.45988
Epoch 574/2000
147/147 [=============================


Epoch 00625: val_loss did not improve from 41.45988
Epoch 626/2000
147/147 [==============================] - 69s 472ms/step - loss: 39.3964 - root_mean_squared_error: 6.2719 - mean_absolute_error: 4.9084 - val_loss: 57.3938 - val_root_mean_squared_error: 7.5759 - val_mean_absolute_error: 5.7185

Epoch 00626: val_loss did not improve from 41.45988
Epoch 627/2000
147/147 [==============================] - 69s 467ms/step - loss: 44.0134 - root_mean_squared_error: 6.6236 - mean_absolute_error: 5.2017 - val_loss: 65.2734 - val_root_mean_squared_error: 8.0792 - val_mean_absolute_error: 6.2167

Epoch 00627: val_loss did not improve from 41.45988
Epoch 628/2000
147/147 [==============================] - 69s 471ms/step - loss: 44.1775 - root_mean_squared_error: 6.6439 - mean_absolute_error: 5.3804 - val_loss: 57.6641 - val_root_mean_squared_error: 7.5937 - val_mean_absolute_error: 5.8850

Epoch 00628: val_loss did not improve from 41.45988
Epoch 629/2000
147/147 [=============================

In [21]:
%%time

epochs = 2000

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs, 
                    callbacks=[checkpointer, earlystopper, reduce_lr])

Epoch 1/2000
143/143 [==============================] - 86s 597ms/step - loss: 279.4282 - root_mean_squared_error: 16.2813 - mean_absolute_error: 13.3851 - val_loss: 125.2096 - val_root_mean_squared_error: 11.1897 - val_mean_absolute_error: 9.2323

Epoch 00001: val_loss improved from inf to 125.20957, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/CNN_regressor_01.hdf5
Epoch 2/2000
143/143 [==============================] - 85s 598ms/step - loss: 100.3308 - root_mean_squared_error: 10.0047 - mean_absolute_error: 8.3252 - val_loss: 109.5805 - val_root_mean_squared_error: 10.4681 - val_mean_absolute_error: 8.7215

Epoch 00002: val_loss improved from 125.20957 to 109.58045, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/CNN_regressor_01.hdf5
Epoch 3/2000
143/143 [==============================] - 84s 592ms/step - loss: 101.5108 - root_mean_squared_error: 10.0737 - mean_absolute_error: 8.4047 - val_loss

143/143 [==============================] - 89s 623ms/step - loss: 89.8423 - root_mean_squared_error: 9.4611 - mean_absolute_error: 7.8222 - val_loss: 82.3853 - val_root_mean_squared_error: 9.0766 - val_mean_absolute_error: 7.4010

Epoch 00025: val_loss improved from 84.11166 to 82.38532, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/CNN_regressor_01.hdf5
Epoch 26/2000
143/143 [==============================] - 87s 606ms/step - loss: 73.9683 - root_mean_squared_error: 8.5938 - mean_absolute_error: 6.9394 - val_loss: 80.3648 - val_root_mean_squared_error: 8.9646 - val_mean_absolute_error: 7.4243

Epoch 00026: val_loss improved from 82.38532 to 80.36481, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/CNN_regressor_01.hdf5
Epoch 27/2000
143/143 [==============================] - 89s 625ms/step - loss: 91.4411 - root_mean_squared_error: 9.5581 - mean_absolute_error: 7.9797 - val_loss: 86.2020 - val_root


Epoch 00077: val_loss did not improve from 63.12808
Epoch 78/2000
143/143 [==============================] - 87s 610ms/step - loss: 72.6780 - root_mean_squared_error: 8.5185 - mean_absolute_error: 6.9134 - val_loss: 74.6236 - val_root_mean_squared_error: 8.6385 - val_mean_absolute_error: 7.1344

Epoch 00078: val_loss did not improve from 63.12808
Epoch 79/2000
143/143 [==============================] - 90s 630ms/step - loss: 73.6307 - root_mean_squared_error: 8.5765 - mean_absolute_error: 6.9986 - val_loss: 76.8785 - val_root_mean_squared_error: 8.7680 - val_mean_absolute_error: 7.2335

Epoch 00079: val_loss did not improve from 63.12808
Epoch 80/2000
143/143 [==============================] - 88s 618ms/step - loss: 66.3762 - root_mean_squared_error: 8.1406 - mean_absolute_error: 6.6636 - val_loss: 77.6888 - val_root_mean_squared_error: 8.8141 - val_mean_absolute_error: 7.5053

Epoch 00080: val_loss did not improve from 63.12808
Epoch 81/2000
143/143 [==============================] -


Epoch 00132: val_loss did not improve from 63.12808
Epoch 133/2000
143/143 [==============================] - 89s 620ms/step - loss: 63.8766 - root_mean_squared_error: 7.9906 - mean_absolute_error: 6.4591 - val_loss: 82.6608 - val_root_mean_squared_error: 9.0918 - val_mean_absolute_error: 7.4208

Epoch 00133: val_loss did not improve from 63.12808
Epoch 134/2000
143/143 [==============================] - 90s 627ms/step - loss: 60.4058 - root_mean_squared_error: 7.7690 - mean_absolute_error: 6.2592 - val_loss: 64.3208 - val_root_mean_squared_error: 8.0200 - val_mean_absolute_error: 6.5729

Epoch 00134: val_loss did not improve from 63.12808
Epoch 135/2000
143/143 [==============================] - 88s 618ms/step - loss: 66.1492 - root_mean_squared_error: 8.1267 - mean_absolute_error: 6.5570 - val_loss: 88.9866 - val_root_mean_squared_error: 9.4333 - val_mean_absolute_error: 7.8944

Epoch 00135: val_loss did not improve from 63.12808
Epoch 136/2000
143/143 [=============================

143/143 [==============================] - 88s 615ms/step - loss: 54.5504 - root_mean_squared_error: 7.3847 - mean_absolute_error: 6.0089 - val_loss: 66.7845 - val_root_mean_squared_error: 8.1722 - val_mean_absolute_error: 6.4370

Epoch 00160: val_loss did not improve from 61.85507
Epoch 161/2000
143/143 [==============================] - 91s 638ms/step - loss: 62.4492 - root_mean_squared_error: 7.9006 - mean_absolute_error: 6.4182 - val_loss: 108.1886 - val_root_mean_squared_error: 10.4014 - val_mean_absolute_error: 8.6722

Epoch 00161: val_loss did not improve from 61.85507
Epoch 162/2000
143/143 [==============================] - 90s 628ms/step - loss: 59.3878 - root_mean_squared_error: 7.7043 - mean_absolute_error: 6.2287 - val_loss: 70.8955 - val_root_mean_squared_error: 8.4199 - val_mean_absolute_error: 7.0346

Epoch 00162: val_loss did not improve from 61.85507
Epoch 163/2000
143/143 [==============================] - 89s 621ms/step - loss: 59.9455 - root_mean_squared_error: 7.7

...


Epoch 00378: val_loss did not improve from 52.43313
Epoch 379/2000
143/143 [==============================] - 89s 623ms/step - loss: 50.7136 - root_mean_squared_error: 7.1125 - mean_absolute_error: 5.6872 - val_loss: 59.6907 - val_root_mean_squared_error: 7.7260 - val_mean_absolute_error: 6.3627

Epoch 00379: val_loss did not improve from 52.43313
Epoch 380/2000
143/143 [==============================] - 89s 625ms/step - loss: 48.1515 - root_mean_squared_error: 6.9305 - mean_absolute_error: 5.5374 - val_loss: 60.8400 - val_root_mean_squared_error: 7.8000 - val_mean_absolute_error: 6.1673

Epoch 00380: val_loss did not improve from 52.43313
Epoch 381/2000
143/143 [==============================] - 90s 627ms/step - loss: 49.6876 - root_mean_squared_error: 7.0454 - mean_absolute_error: 5.7163 - val_loss: 69.3961 - val_root_mean_squared_error: 8.3304 - val_mean_absolute_error: 6.7188

Epoch 00381: val_loss did not improve from 52.43313
Epoch 382/2000
143/143 [=============================

143/143 [==============================] - 88s 615ms/step - loss: 48.1234 - root_mean_squared_error: 6.9340 - mean_absolute_error: 5.5232 - val_loss: 58.7630 - val_root_mean_squared_error: 7.6657 - val_mean_absolute_error: 6.1006

Epoch 00406: val_loss did not improve from 52.43313
Epoch 407/2000
143/143 [==============================] - 91s 633ms/step - loss: 51.7177 - root_mean_squared_error: 7.1873 - mean_absolute_error: 5.8510 - val_loss: 57.9693 - val_root_mean_squared_error: 7.6138 - val_mean_absolute_error: 6.0963

Epoch 00407: val_loss did not improve from 52.43313
Epoch 408/2000
143/143 [==============================] - 90s 630ms/step - loss: 45.2071 - root_mean_squared_error: 6.7195 - mean_absolute_error: 5.4343 - val_loss: 64.2619 - val_root_mean_squared_error: 8.0163 - val_mean_absolute_error: 6.2177

Epoch 00408: val_loss did not improve from 52.43313
Epoch 409/2000
143/143 [==============================] - 89s 620ms/step - loss: 48.5202 - root_mean_squared_error: 6.964


Epoch 00433: val_loss did not improve from 52.43313
Epoch 434/2000
143/143 [==============================] - 90s 631ms/step - loss: 46.3432 - root_mean_squared_error: 6.8029 - mean_absolute_error: 5.4785 - val_loss: 64.3471 - val_root_mean_squared_error: 8.0217 - val_mean_absolute_error: 6.7204

Epoch 00434: val_loss did not improve from 52.43313
Epoch 435/2000
143/143 [==============================] - 90s 630ms/step - loss: 52.1919 - root_mean_squared_error: 7.2182 - mean_absolute_error: 5.7960 - val_loss: 62.0539 - val_root_mean_squared_error: 7.8774 - val_mean_absolute_error: 6.3092

Epoch 00435: val_loss did not improve from 52.43313
Epoch 436/2000
143/143 [==============================] - 89s 621ms/step - loss: 46.8472 - root_mean_squared_error: 6.8339 - mean_absolute_error: 5.5346 - val_loss: 59.8626 - val_root_mean_squared_error: 7.7371 - val_mean_absolute_error: 6.2072

Epoch 00436: val_loss did not improve from 52.43313
Epoch 437/2000
143/143 [=============================

143/143 [==============================] - 89s 623ms/step - loss: 47.0169 - root_mean_squared_error: 6.8538 - mean_absolute_error: 5.5345 - val_loss: 58.9815 - val_root_mean_squared_error: 7.6799 - val_mean_absolute_error: 6.2691

Epoch 00461: val_loss did not improve from 52.43313
Epoch 462/2000
143/143 [==============================] - 85s 595ms/step - loss: 50.9829 - root_mean_squared_error: 7.1364 - mean_absolute_error: 5.7611 - val_loss: 61.5617 - val_root_mean_squared_error: 7.8461 - val_mean_absolute_error: 6.2732

Epoch 00462: val_loss did not improve from 52.43313
Epoch 463/2000
143/143 [==============================] - 85s 592ms/step - loss: 46.9969 - root_mean_squared_error: 6.8518 - mean_absolute_error: 5.4844 - val_loss: 67.6572 - val_root_mean_squared_error: 8.2254 - val_mean_absolute_error: 6.6349

Epoch 00463: val_loss did not improve from 52.43313
Epoch 464/2000
143/143 [==============================] - 87s 608ms/step - loss: 48.1028 - root_mean_squared_error: 6.934


Epoch 00488: val_loss did not improve from 52.43313
Epoch 489/2000
143/143 [==============================] - 90s 627ms/step - loss: 53.3000 - root_mean_squared_error: 7.2993 - mean_absolute_error: 5.7820 - val_loss: 60.6261 - val_root_mean_squared_error: 7.7863 - val_mean_absolute_error: 6.3368

Epoch 00489: val_loss did not improve from 52.43313
Epoch 490/2000
143/143 [==============================] - 89s 626ms/step - loss: 49.4817 - root_mean_squared_error: 7.0331 - mean_absolute_error: 5.6485 - val_loss: 67.0024 - val_root_mean_squared_error: 8.1855 - val_mean_absolute_error: 6.3759

Epoch 00490: val_loss did not improve from 52.43313
Epoch 491/2000
143/143 [==============================] - 91s 634ms/step - loss: 51.1589 - root_mean_squared_error: 7.1505 - mean_absolute_error: 5.5952 - val_loss: 58.4885 - val_root_mean_squared_error: 7.6478 - val_mean_absolute_error: 6.0661

Epoch 00491: val_loss did not improve from 52.43313
Epoch 492/2000
143/143 [=============================

# 3. ResNet

In [68]:
def resnet_model():
    """ Returns the ResNet model from Ismail Fawaz et al. (2019). """
    n_feature_maps = 64

    input_layer = keras.layers.Input(input_shape)

    # BLOCK 1

    conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # expand channels for the sum
    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_1 = keras.layers.add([shortcut_y, conv_z])
    output_block_1 = keras.layers.Activation('relu')(output_block_1)

    # BLOCK 2

    conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # expand channels for the sum
    shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
    shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

    output_block_2 = keras.layers.add([shortcut_y, conv_z])
    output_block_2 = keras.layers.Activation('relu')(output_block_2)

    # BLOCK 3

    conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
    conv_x = keras.layers.BatchNormalization()(conv_x)
    conv_x = keras.layers.Activation('relu')(conv_x)

    conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
    conv_y = keras.layers.BatchNormalization()(conv_y)
    conv_y = keras.layers.Activation('relu')(conv_y)

    conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
    conv_z = keras.layers.BatchNormalization()(conv_z)

    # no need to expand channels because they are equal
    shortcut_y = keras.layers.BatchNormalization()(output_block_2)

    output_block_3 = keras.layers.add([shortcut_y, conv_z])
    output_block_3 = keras.layers.Activation('relu')(output_block_3)

    # FINAL

    pooling_layer = tf.keras.layers.AveragePooling1D(pool_size=50)(output_block_3)
    flat_layer = tf.keras.layers.Flatten()(pooling_layer)
    dense_layer = tf.keras.layers.Dense(128, activation='relu')(flat_layer)

    output_layer = keras.layers.Dense(1)(dense_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model

# def resnet_model():
#     """ Returns the ResNet model from Ismail Fawaz et al. (2019). """
#     n_feature_maps = 64

#     input_layer = keras.layers.Input(input_shape)

#     # BLOCK 1

#     conv_x = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=8, padding='same')(input_layer)
#     conv_x = keras.layers.BatchNormalization()(conv_x)
#     conv_x = keras.layers.Activation('relu')(conv_x)

#     conv_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=5, padding='same')(conv_x)
#     conv_y = keras.layers.BatchNormalization()(conv_y)
#     conv_y = keras.layers.Activation('relu')(conv_y)

#     conv_z = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=3, padding='same')(conv_y)
#     conv_z = keras.layers.BatchNormalization()(conv_z)

#     # expand channels for the sum
#     shortcut_y = keras.layers.Conv1D(filters=n_feature_maps, kernel_size=1, padding='same')(input_layer)
#     shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

#     output_block_1 = keras.layers.add([shortcut_y, conv_z])
#     output_block_1 = keras.layers.Activation('relu')(output_block_1)

#     # BLOCK 2

#     conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_1)
#     conv_x = keras.layers.BatchNormalization()(conv_x)
#     conv_x = keras.layers.Activation('relu')(conv_x)

#     conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
#     conv_y = keras.layers.BatchNormalization()(conv_y)
#     conv_y = keras.layers.Activation('relu')(conv_y)

#     conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
#     conv_z = keras.layers.BatchNormalization()(conv_z)

#     # expand channels for the sum
#     shortcut_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=1, padding='same')(output_block_1)
#     shortcut_y = keras.layers.BatchNormalization()(shortcut_y)

#     output_block_2 = keras.layers.add([shortcut_y, conv_z])
#     output_block_2 = keras.layers.Activation('relu')(output_block_2)

#     # BLOCK 3

#     conv_x = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=8, padding='same')(output_block_2)
#     conv_x = keras.layers.BatchNormalization()(conv_x)
#     conv_x = keras.layers.Activation('relu')(conv_x)

#     conv_y = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=5, padding='same')(conv_x)
#     conv_y = keras.layers.BatchNormalization()(conv_y)
#     conv_y = keras.layers.Activation('relu')(conv_y)

#     conv_z = keras.layers.Conv1D(filters=n_feature_maps * 2, kernel_size=3, padding='same')(conv_y)
#     conv_z = keras.layers.BatchNormalization()(conv_z)

#     # no need to expand channels because they are equal
#     shortcut_y = keras.layers.BatchNormalization()(output_block_2)

#     output_block_3 = keras.layers.add([shortcut_y, conv_z])
#     output_block_3 = keras.layers.Activation('relu')(output_block_3)

#     # FINAL

#     gap_layer = keras.layers.GlobalAveragePooling1D()(output_block_3)

#     output_layer = keras.layers.Dense(1)(gap_layer)

#     model = keras.models.Model(inputs=input_layer, outputs=output_layer)

#     return model

In [69]:
model = resnet_model()

optimizer = Adam(learning_rate=0.01)    
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

# ResNet_regressor_01: MSE, Adam, N_average=30, 1500 epochs, ES=250, RLR=50, gaussian=0.01
output_filename = 'ResNet_regressor_FINAL_LOCAL'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=300, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=0.0001, verbose=1)

In [70]:
%%time

epochs = 1500

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs, 
                    callbacks=[checkpointer, earlystopper, reduce_lr])

Epoch 1/1500
147/147 [==============================] - 80s 524ms/step - loss: 162.8885 - root_mean_squared_error: 12.5711 - mean_absolute_error: 10.1963 - val_loss: 4439.4014 - val_root_mean_squared_error: 66.6288 - val_mean_absolute_error: 44.3474

Epoch 00001: val_loss improved from inf to 4439.40137, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/ResNet_regressor_FINAL_LOCAL.hdf5
Epoch 2/1500
147/147 [==============================] - 77s 527ms/step - loss: 101.0106 - root_mean_squared_error: 10.0488 - mean_absolute_error: 8.2213 - val_loss: 200.0082 - val_root_mean_squared_error: 14.1424 - val_mean_absolute_error: 10.3263

Epoch 00002: val_loss improved from 4439.40137 to 200.00819, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/ResNet_regressor_FINAL_LOCAL.hdf5
Epoch 3/1500
147/147 [==============================] - 77s 523ms/step - loss: 94.5808 - root_mean_squared_error: 9.7198 - mean_absolu

147/147 [==============================] - 78s 531ms/step - loss: 59.4175 - root_mean_squared_error: 7.7053 - mean_absolute_error: 6.2468 - val_loss: 55.0797 - val_root_mean_squared_error: 7.4216 - val_mean_absolute_error: 5.7998

Epoch 00050: val_loss improved from 60.25277 to 55.07965, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/ResNet_regressor_FINAL_LOCAL.hdf5
Epoch 51/1500
147/147 [==============================] - 78s 530ms/step - loss: 60.4853 - root_mean_squared_error: 7.7740 - mean_absolute_error: 6.2706 - val_loss: 63.8879 - val_root_mean_squared_error: 7.9930 - val_mean_absolute_error: 6.2887

Epoch 00051: val_loss did not improve from 55.07965
Epoch 52/1500
147/147 [==============================] - 78s 529ms/step - loss: 59.3856 - root_mean_squared_error: 7.7030 - mean_absolute_error: 6.1881 - val_loss: 69.9053 - val_root_mean_squared_error: 8.3609 - val_mean_absolute_error: 6.9698

Epoch 00052: val_loss did not improve from 5


Epoch 00076: val_loss did not improve from 44.45996
Epoch 77/1500
147/147 [==============================] - 79s 537ms/step - loss: 59.5014 - root_mean_squared_error: 7.7113 - mean_absolute_error: 6.1254 - val_loss: 61.5360 - val_root_mean_squared_error: 7.8445 - val_mean_absolute_error: 6.0492

Epoch 00077: val_loss did not improve from 44.45996
Epoch 78/1500
147/147 [==============================] - 79s 537ms/step - loss: 49.8932 - root_mean_squared_error: 7.0517 - mean_absolute_error: 5.7092 - val_loss: 58.4688 - val_root_mean_squared_error: 7.6465 - val_mean_absolute_error: 6.1040

Epoch 00078: val_loss did not improve from 44.45996
Epoch 79/1500
147/147 [==============================] - 79s 535ms/step - loss: 51.7994 - root_mean_squared_error: 7.1934 - mean_absolute_error: 5.9239 - val_loss: 69.9308 - val_root_mean_squared_error: 8.3625 - val_mean_absolute_error: 6.5537

Epoch 00079: val_loss did not improve from 44.45996
Epoch 80/1500
147/147 [==============================] -

147/147 [==============================] - 78s 534ms/step - loss: 48.6432 - root_mean_squared_error: 6.9678 - mean_absolute_error: 5.5683 - val_loss: 52.2700 - val_root_mean_squared_error: 7.2298 - val_mean_absolute_error: 5.5611

Epoch 00131: val_loss did not improve from 44.45996
Epoch 132/1500
147/147 [==============================] - 79s 536ms/step - loss: 52.1619 - root_mean_squared_error: 7.2155 - mean_absolute_error: 5.6984 - val_loss: 71.2680 - val_root_mean_squared_error: 8.4420 - val_mean_absolute_error: 6.5943

Epoch 00132: val_loss did not improve from 44.45996
Epoch 133/1500
147/147 [==============================] - 78s 534ms/step - loss: 50.1336 - root_mean_squared_error: 7.0789 - mean_absolute_error: 5.6380 - val_loss: 55.4848 - val_root_mean_squared_error: 7.4488 - val_mean_absolute_error: 5.7503

Epoch 00133: val_loss did not improve from 44.45996
Epoch 134/1500
147/147 [==============================] - 79s 534ms/step - loss: 49.9880 - root_mean_squared_error: 7.059


Epoch 00185: val_loss did not improve from 43.51994
Epoch 186/1500
147/147 [==============================] - 79s 539ms/step - loss: 50.7901 - root_mean_squared_error: 7.1220 - mean_absolute_error: 5.6312 - val_loss: 47.2393 - val_root_mean_squared_error: 6.8731 - val_mean_absolute_error: 5.3766

Epoch 00186: val_loss did not improve from 43.51994
Epoch 187/1500
147/147 [==============================] - 78s 533ms/step - loss: 43.5115 - root_mean_squared_error: 6.5887 - mean_absolute_error: 5.2881 - val_loss: 53.9465 - val_root_mean_squared_error: 7.3448 - val_mean_absolute_error: 5.5892

Epoch 00187: val_loss did not improve from 43.51994
Epoch 188/1500
147/147 [==============================] - 79s 535ms/step - loss: 47.4989 - root_mean_squared_error: 6.8894 - mean_absolute_error: 5.5028 - val_loss: 52.2802 - val_root_mean_squared_error: 7.2305 - val_mean_absolute_error: 5.7777

Epoch 00188: val_loss did not improve from 43.51994
Epoch 189/1500
147/147 [=============================

...


Epoch 00541: val_loss did not improve from 37.04778
Epoch 542/1500
147/147 [==============================] - 78s 529ms/step - loss: 40.2832 - root_mean_squared_error: 6.3444 - mean_absolute_error: 5.0982 - val_loss: 57.9369 - val_root_mean_squared_error: 7.6116 - val_mean_absolute_error: 5.9552

Epoch 00542: val_loss did not improve from 37.04778
Epoch 543/1500
147/147 [==============================] - 78s 529ms/step - loss: 42.7496 - root_mean_squared_error: 6.5339 - mean_absolute_error: 5.2306 - val_loss: 58.6742 - val_root_mean_squared_error: 7.6599 - val_mean_absolute_error: 5.8096

Epoch 00543: val_loss did not improve from 37.04778
Epoch 544/1500
147/147 [==============================] - 78s 532ms/step - loss: 36.3313 - root_mean_squared_error: 6.0231 - mean_absolute_error: 4.7446 - val_loss: 47.7623 - val_root_mean_squared_error: 6.9110 - val_mean_absolute_error: 5.2205

Epoch 00544: val_loss did not improve from 37.04778
Epoch 545/1500
147/147 [=============================


Epoch 00596: val_loss did not improve from 37.04778
Epoch 597/1500
147/147 [==============================] - 88s 599ms/step - loss: 42.1544 - root_mean_squared_error: 6.4909 - mean_absolute_error: 5.2064 - val_loss: 52.2237 - val_root_mean_squared_error: 7.2266 - val_mean_absolute_error: 5.5593

Epoch 00597: val_loss did not improve from 37.04778
Epoch 598/1500
147/147 [==============================] - 87s 591ms/step - loss: 40.3455 - root_mean_squared_error: 6.3473 - mean_absolute_error: 5.0087 - val_loss: 58.7698 - val_root_mean_squared_error: 7.6661 - val_mean_absolute_error: 5.7239

Epoch 00598: val_loss did not improve from 37.04778
Epoch 599/1500
147/147 [==============================] - 105s 717ms/step - loss: 41.2503 - root_mean_squared_error: 6.4214 - mean_absolute_error: 5.1792 - val_loss: 53.2599 - val_root_mean_squared_error: 7.2979 - val_mean_absolute_error: 5.6935

Epoch 00599: val_loss did not improve from 37.04778
Epoch 600/1500
147/147 [============================


Epoch 00651: val_loss did not improve from 37.04778
Epoch 652/1500
147/147 [==============================] - 137s 932ms/step - loss: 40.5056 - root_mean_squared_error: 6.3614 - mean_absolute_error: 5.0248 - val_loss: 52.2262 - val_root_mean_squared_error: 7.2268 - val_mean_absolute_error: 5.4852

Epoch 00652: val_loss did not improve from 37.04778
Epoch 653/1500
147/147 [==============================] - 136s 928ms/step - loss: 42.0427 - root_mean_squared_error: 6.4677 - mean_absolute_error: 5.0571 - val_loss: 57.5233 - val_root_mean_squared_error: 7.5844 - val_mean_absolute_error: 5.7972

Epoch 00653: val_loss did not improve from 37.04778
Epoch 654/1500
147/147 [==============================] - 135s 923ms/step - loss: 41.1884 - root_mean_squared_error: 6.4158 - mean_absolute_error: 5.1390 - val_loss: 51.1884 - val_root_mean_squared_error: 7.1546 - val_mean_absolute_error: 5.5212

Epoch 00654: val_loss did not improve from 37.04778
Epoch 655/1500
147/147 [==========================


Epoch 00706: val_loss did not improve from 37.04778
Epoch 707/1500
147/147 [==============================] - 80s 544ms/step - loss: 43.9748 - root_mean_squared_error: 6.6285 - mean_absolute_error: 5.2907 - val_loss: 48.5529 - val_root_mean_squared_error: 6.9680 - val_mean_absolute_error: 5.4975

Epoch 00707: val_loss did not improve from 37.04778
Epoch 708/1500
147/147 [==============================] - 80s 545ms/step - loss: 38.9815 - root_mean_squared_error: 6.2356 - mean_absolute_error: 4.9259 - val_loss: 46.6716 - val_root_mean_squared_error: 6.8317 - val_mean_absolute_error: 5.1474

Epoch 00708: val_loss did not improve from 37.04778
Epoch 709/1500
147/147 [==============================] - 80s 543ms/step - loss: 43.7868 - root_mean_squared_error: 6.6084 - mean_absolute_error: 5.3284 - val_loss: 48.8154 - val_root_mean_squared_error: 6.9868 - val_mean_absolute_error: 5.4005

Epoch 00709: val_loss did not improve from 37.04778
Epoch 710/1500
147/147 [=============================


Epoch 00761: val_loss did not improve from 37.04778
Epoch 762/1500
147/147 [==============================] - 79s 538ms/step - loss: 41.4599 - root_mean_squared_error: 6.4369 - mean_absolute_error: 5.0353 - val_loss: 46.7918 - val_root_mean_squared_error: 6.8405 - val_mean_absolute_error: 5.2427

Epoch 00762: val_loss did not improve from 37.04778
Epoch 763/1500
147/147 [==============================] - 79s 538ms/step - loss: 40.1480 - root_mean_squared_error: 6.3317 - mean_absolute_error: 5.0456 - val_loss: 54.6306 - val_root_mean_squared_error: 7.3913 - val_mean_absolute_error: 5.4899

Epoch 00763: val_loss did not improve from 37.04778
Epoch 764/1500
147/147 [==============================] - 79s 535ms/step - loss: 40.1204 - root_mean_squared_error: 6.3318 - mean_absolute_error: 5.0288 - val_loss: 52.8637 - val_root_mean_squared_error: 7.2707 - val_mean_absolute_error: 5.4002

Epoch 00764: val_loss did not improve from 37.04778
Epoch 765/1500
147/147 [=============================

# 4. Encoder

In [71]:
import tensorflow_addons as tfa

def encoder_model():
    """ Returns the Encoder model from Ismail Fawaz et al. (2019). """
    input_layer = keras.layers.Input(input_shape)

    # conv block -1
    conv1 = keras.layers.Conv1D(filters=128,kernel_size=5,strides=1,padding='same')(input_layer)
    conv1 = tfa.layers.InstanceNormalization()(conv1)
    conv1 = keras.layers.PReLU(shared_axes=[1])(conv1)
    conv1 = keras.layers.Dropout(rate=0.2)(conv1)
    conv1 = keras.layers.MaxPooling1D(pool_size=2)(conv1)
    # conv block -2
    conv2 = keras.layers.Conv1D(filters=256,kernel_size=11,strides=1,padding='same')(conv1)
    conv2 = tfa.layers.InstanceNormalization()(conv2)
    conv2 = keras.layers.PReLU(shared_axes=[1])(conv2)
    conv2 = keras.layers.Dropout(rate=0.2)(conv2)
    conv2 = keras.layers.MaxPooling1D(pool_size=2)(conv2)
    # conv block -3
    conv3 = keras.layers.Conv1D(filters=512,kernel_size=21,strides=1,padding='same')(conv2)
    conv3 = tfa.layers.InstanceNormalization()(conv3)
    conv3 = keras.layers.PReLU(shared_axes=[1])(conv3)
    conv3 = keras.layers.Dropout(rate=0.2)(conv3)
    # split for attention
    attention_data = keras.layers.Lambda(lambda x: x[:,:,:256])(conv3)
    attention_softmax = keras.layers.Lambda(lambda x: x[:,:,256:])(conv3)
    # attention mechanism
    attention_softmax = keras.layers.Softmax()(attention_softmax)
    multiply_layer = keras.layers.Multiply()([attention_softmax,attention_data])
    # last layer
    dense_layer = keras.layers.Dense(units=256,activation='sigmoid')(multiply_layer)
    dense_layer = tfa.layers.InstanceNormalization()(dense_layer)
    # output layer
    flatten_layer = keras.layers.Flatten()(dense_layer)
    output_layer = keras.layers.Dense(1)(flatten_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model

/Users/bbruns/anaconda3/envs/mne/lib/python3.8/site-packages/tensorflow_addons/utils/ensure_tf_install.py:54: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.2.0 and strictly below 2.4.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.4.1 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You can find the compatibility matrix in TensorFlow Addon's readme:
https://github.com/tensorflow/addons
  warnings.warn(


In [72]:
model = encoder_model()

# optimizer = Adam(learning_rate=0.00001) 
optimizer = Adam(learning_rate=0.01)
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

output_filename = f'Encoder_regressor_CROSSVALIDATION_{}'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=300, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=0.00001, verbose=1)

In [73]:
%%time



Epoch 1/1000
147/147 [==============================] - 81s 535ms/step - loss: 179.5678 - root_mean_squared_error: 13.1139 - mean_absolute_error: 10.3594 - val_loss: 105.3348 - val_root_mean_squared_error: 10.2633 - val_mean_absolute_error: 8.4479

Epoch 00001: val_loss improved from inf to 105.33485, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Encoder_regressor_FINAL_LOCAL.hdf5
Epoch 2/1000
147/147 [==============================] - 78s 530ms/step - loss: 96.0447 - root_mean_squared_error: 9.7891 - mean_absolute_error: 7.9585 - val_loss: 87.3716 - val_root_mean_squared_error: 9.3473 - val_mean_absolute_error: 7.4615

Epoch 00002: val_loss improved from 105.33485 to 87.37163, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Encoder_regressor_FINAL_LOCAL.hdf5
Epoch 3/1000
147/147 [==============================] - 79s 535ms/step - loss: 98.8634 - root_mean_squared_error: 9.9323 - mean_absolute_error

147/147 [==============================] - 120s 816ms/step - loss: 71.2160 - root_mean_squared_error: 8.4332 - mean_absolute_error: 6.8177 - val_loss: 96.1386 - val_root_mean_squared_error: 9.8050 - val_mean_absolute_error: 7.4689

Epoch 00052: val_loss did not improve from 61.95862
Epoch 53/1000
147/147 [==============================] - 121s 824ms/step - loss: 62.4550 - root_mean_squared_error: 7.9013 - mean_absolute_error: 6.3592 - val_loss: 64.5238 - val_root_mean_squared_error: 8.0327 - val_mean_absolute_error: 6.5622

Epoch 00053: val_loss did not improve from 61.95862
Epoch 54/1000
147/147 [==============================] - 133s 904ms/step - loss: 69.5317 - root_mean_squared_error: 8.3327 - mean_absolute_error: 6.7688 - val_loss: 85.0919 - val_root_mean_squared_error: 9.2245 - val_mean_absolute_error: 7.7339

Epoch 00054: val_loss did not improve from 61.95862
Epoch 55/1000
147/147 [==============================] - 129s 877ms/step - loss: 61.6827 - root_mean_squared_error: 7.85


Epoch 00105: val_loss did not improve from 48.28324
Epoch 106/1000
147/147 [==============================] - 77s 526ms/step - loss: 54.1806 - root_mean_squared_error: 7.3559 - mean_absolute_error: 5.8721 - val_loss: 58.8868 - val_root_mean_squared_error: 7.6738 - val_mean_absolute_error: 6.0181

Epoch 00106: val_loss did not improve from 48.28324
Epoch 107/1000
147/147 [==============================] - 77s 523ms/step - loss: 50.4202 - root_mean_squared_error: 7.0976 - mean_absolute_error: 5.6667 - val_loss: 54.6446 - val_root_mean_squared_error: 7.3922 - val_mean_absolute_error: 6.0851

Epoch 00107: val_loss did not improve from 48.28324
Epoch 108/1000
147/147 [==============================] - 77s 525ms/step - loss: 49.7864 - root_mean_squared_error: 7.0526 - mean_absolute_error: 5.6101 - val_loss: 64.9751 - val_root_mean_squared_error: 8.0607 - val_mean_absolute_error: 6.6763

Epoch 00108: val_loss did not improve from 48.28324
Epoch 109/1000
147/147 [=============================

147/147 [==============================] - 77s 524ms/step - loss: 42.3916 - root_mean_squared_error: 6.5080 - mean_absolute_error: 5.1316 - val_loss: 63.8131 - val_root_mean_squared_error: 7.9883 - val_mean_absolute_error: 6.2658

Epoch 00133: val_loss did not improve from 48.28324
Epoch 134/1000
147/147 [==============================] - 77s 521ms/step - loss: 45.4238 - root_mean_squared_error: 6.7336 - mean_absolute_error: 5.2830 - val_loss: 46.2931 - val_root_mean_squared_error: 6.8039 - val_mean_absolute_error: 5.6892

Epoch 00134: val_loss improved from 48.28324 to 46.29308, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Encoder_regressor_FINAL_LOCAL.hdf5
Epoch 135/1000
147/147 [==============================] - 77s 524ms/step - loss: 41.1656 - root_mean_squared_error: 6.4075 - mean_absolute_error: 5.1254 - val_loss: 54.5881 - val_root_mean_squared_error: 7.3884 - val_mean_absolute_error: 5.9110

Epoch 00135: val_loss did not improve fro

147/147 [==============================] - 77s 524ms/step - loss: 44.1082 - root_mean_squared_error: 6.6344 - mean_absolute_error: 5.2771 - val_loss: 54.8767 - val_root_mean_squared_error: 7.4079 - val_mean_absolute_error: 6.0505

Epoch 00187: val_loss did not improve from 46.29308
Epoch 188/1000
147/147 [==============================] - 77s 525ms/step - loss: 36.7680 - root_mean_squared_error: 6.0611 - mean_absolute_error: 4.7772 - val_loss: 57.4687 - val_root_mean_squared_error: 7.5808 - val_mean_absolute_error: 6.1478

Epoch 00188: val_loss did not improve from 46.29308
Epoch 189/1000
147/147 [==============================] - 78s 529ms/step - loss: 38.8902 - root_mean_squared_error: 6.2351 - mean_absolute_error: 4.9345 - val_loss: 45.9978 - val_root_mean_squared_error: 6.7822 - val_mean_absolute_error: 5.4055

Epoch 00189: val_loss improved from 46.29308 to 45.99777, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Encoder_regressor_FINAL_

...


Epoch 00350: val_loss did not improve from 44.44696
Epoch 351/1000
147/147 [==============================] - 78s 527ms/step - loss: 32.1002 - root_mean_squared_error: 5.6632 - mean_absolute_error: 4.3785 - val_loss: 72.6258 - val_root_mean_squared_error: 8.5221 - val_mean_absolute_error: 6.8993

Epoch 00351: val_loss did not improve from 44.44696
Epoch 352/1000
147/147 [==============================] - 77s 526ms/step - loss: 29.9045 - root_mean_squared_error: 5.4670 - mean_absolute_error: 4.3197 - val_loss: 65.8524 - val_root_mean_squared_error: 8.1149 - val_mean_absolute_error: 6.4868

Epoch 00352: val_loss did not improve from 44.44696
Epoch 353/1000
147/147 [==============================] - 77s 526ms/step - loss: 34.6139 - root_mean_squared_error: 5.8790 - mean_absolute_error: 4.6958 - val_loss: 65.3002 - val_root_mean_squared_error: 8.0809 - val_mean_absolute_error: 6.4599

Epoch 00353: val_loss did not improve from 44.44696
Epoch 354/1000
147/147 [=============================

147/147 [==============================] - 78s 529ms/step - loss: 31.8333 - root_mean_squared_error: 5.6387 - mean_absolute_error: 4.3644 - val_loss: 66.8777 - val_root_mean_squared_error: 8.1779 - val_mean_absolute_error: 6.5184

Epoch 00378: val_loss did not improve from 44.44696
Epoch 379/1000
147/147 [==============================] - 78s 530ms/step - loss: 31.8312 - root_mean_squared_error: 5.6398 - mean_absolute_error: 4.4985 - val_loss: 60.5027 - val_root_mean_squared_error: 7.7783 - val_mean_absolute_error: 6.2596

Epoch 00379: val_loss did not improve from 44.44696
Epoch 380/1000
147/147 [==============================] - 78s 529ms/step - loss: 30.3662 - root_mean_squared_error: 5.5095 - mean_absolute_error: 4.3841 - val_loss: 55.6641 - val_root_mean_squared_error: 7.4608 - val_mean_absolute_error: 5.8849

Epoch 00380: val_loss did not improve from 44.44696
Epoch 381/1000
147/147 [==============================] - 78s 530ms/step - loss: 31.3180 - root_mean_squared_error: 5.593


Epoch 00405: val_loss did not improve from 44.44696
Epoch 406/1000
147/147 [==============================] - 79s 537ms/step - loss: 34.7705 - root_mean_squared_error: 5.8932 - mean_absolute_error: 4.5873 - val_loss: 60.8688 - val_root_mean_squared_error: 7.8018 - val_mean_absolute_error: 6.3715

Epoch 00406: val_loss did not improve from 44.44696
Epoch 407/1000
147/147 [==============================] - 80s 543ms/step - loss: 30.4958 - root_mean_squared_error: 5.5171 - mean_absolute_error: 4.3622 - val_loss: 52.7336 - val_root_mean_squared_error: 7.2618 - val_mean_absolute_error: 5.9313

Epoch 00407: val_loss did not improve from 44.44696

Epoch 00407: ReduceLROnPlateau reducing learning rate to 0.00015624999650754035.
Epoch 408/1000
147/147 [==============================] - 79s 540ms/step - loss: 28.7875 - root_mean_squared_error: 5.3591 - mean_absolute_error: 4.2461 - val_loss: 58.9570 - val_root_mean_squared_error: 7.6783 - val_mean_absolute_error: 6.0122

Epoch 00408: val_loss d

147/147 [==============================] - 78s 531ms/step - loss: 33.1527 - root_mean_squared_error: 5.7546 - mean_absolute_error: 4.5524 - val_loss: 54.0005 - val_root_mean_squared_error: 7.3485 - val_mean_absolute_error: 6.0467

Epoch 00433: val_loss did not improve from 44.44696
Epoch 434/1000
147/147 [==============================] - 78s 531ms/step - loss: 32.7182 - root_mean_squared_error: 5.7147 - mean_absolute_error: 4.4412 - val_loss: 59.4706 - val_root_mean_squared_error: 7.7117 - val_mean_absolute_error: 6.2052

Epoch 00434: val_loss did not improve from 44.44696
Epoch 435/1000
147/147 [==============================] - 78s 530ms/step - loss: 34.7582 - root_mean_squared_error: 5.8929 - mean_absolute_error: 4.5668 - val_loss: 53.0842 - val_root_mean_squared_error: 7.2859 - val_mean_absolute_error: 5.8545

Epoch 00435: val_loss did not improve from 44.44696
Epoch 436/1000
147/147 [==============================] - 78s 531ms/step - loss: 32.5241 - root_mean_squared_error: 5.701


Epoch 00487: val_loss did not improve from 44.44696
Epoch 488/1000
147/147 [==============================] - 78s 530ms/step - loss: 33.7689 - root_mean_squared_error: 5.8107 - mean_absolute_error: 4.6709 - val_loss: 49.4186 - val_root_mean_squared_error: 7.0298 - val_mean_absolute_error: 5.7192

Epoch 00488: val_loss did not improve from 44.44696
Epoch 489/1000
147/147 [==============================] - 78s 530ms/step - loss: 30.4701 - root_mean_squared_error: 5.5128 - mean_absolute_error: 4.3261 - val_loss: 55.5828 - val_root_mean_squared_error: 7.4554 - val_mean_absolute_error: 6.0425

Epoch 00489: val_loss did not improve from 44.44696
Epoch 490/1000
147/147 [==============================] - 78s 530ms/step - loss: 33.1764 - root_mean_squared_error: 5.7580 - mean_absolute_error: 4.5319 - val_loss: 54.1875 - val_root_mean_squared_error: 7.3612 - val_mean_absolute_error: 5.8958

Epoch 00490: val_loss did not improve from 44.44696
Epoch 491/1000
147/147 [=============================

# 5. Time-CNN

In [11]:
# https://github.com/hfawaz/dl-4-tsc/blob/master/classifiers/cnn.py

def timecnn_model():
    """ Returns the Time-CNN model from Ismail Fawaz et al. (2019). """
    
    padding = 'valid'
    input_layer = keras.layers.Input(input_shape)

    conv1 = keras.layers.Conv1D(filters=6,kernel_size=7,padding=padding,activation='sigmoid')(input_layer)
    conv1 = keras.layers.AveragePooling1D(pool_size=3)(conv1)

    conv2 = keras.layers.Conv1D(filters=12,kernel_size=7,padding=padding,activation='sigmoid')(conv1)
    conv2 = keras.layers.AveragePooling1D(pool_size=3)(conv2)

    flatten_layer = keras.layers.Flatten()(conv2)

    output_layer = keras.layers.Dense(units=1)(flatten_layer)

    model = keras.models.Model(inputs=input_layer, outputs=output_layer)

    return model

In [12]:
model = timecnn_model()

optimizer = Adam(learning_rate=0.01)    
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

# TimeCNN_regressor_01: MSE, Adam, N_average=30, 2000 epochs, ES=250, RLR=50, gaussian=0.01
output_filename = 'TimeCNN_regressor_FINAL_LOCAL'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=300, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=0.0001, verbose=1)

In [13]:
%%time

epochs = 2000

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs, 
                    callbacks=[checkpointer, earlystopper, reduce_lr])

Epoch 1/2000
147/147 [==============================] - 99s 669ms/step - loss: 166.3452 - root_mean_squared_error: 12.7333 - mean_absolute_error: 10.5577 - val_loss: 102.5612 - val_root_mean_squared_error: 10.1272 - val_mean_absolute_error: 8.4824

Epoch 00001: val_loss improved from inf to 102.56120, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/TimeCNN_regressor_FINAL_LOCAL.hdf5
Epoch 2/2000
147/147 [==============================] - 92s 630ms/step - loss: 100.2291 - root_mean_squared_error: 10.0049 - mean_absolute_error: 8.4201 - val_loss: 94.3615 - val_root_mean_squared_error: 9.7140 - val_mean_absolute_error: 7.8813

Epoch 00002: val_loss improved from 102.56120 to 94.36149, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/TimeCNN_regressor_FINAL_LOCAL.hdf5
Epoch 3/2000
147/147 [==============================] - 94s 639ms/step - loss: 96.6228 - root_mean_squared_error: 9.8276 - mean_absolute_err

147/147 [==============================] - 83s 566ms/step - loss: 76.0513 - root_mean_squared_error: 8.7171 - mean_absolute_error: 7.2264 - val_loss: 78.3629 - val_root_mean_squared_error: 8.8523 - val_mean_absolute_error: 7.3002

Epoch 00026: val_loss did not improve from 67.86678
Epoch 27/2000
147/147 [==============================] - 83s 564ms/step - loss: 82.5542 - root_mean_squared_error: 9.0801 - mean_absolute_error: 7.5051 - val_loss: 76.6166 - val_root_mean_squared_error: 8.7531 - val_mean_absolute_error: 7.1179

Epoch 00027: val_loss did not improve from 67.86678
Epoch 28/2000
147/147 [==============================] - 82s 560ms/step - loss: 70.0515 - root_mean_squared_error: 8.3661 - mean_absolute_error: 6.8497 - val_loss: 69.1114 - val_root_mean_squared_error: 8.3133 - val_mean_absolute_error: 6.6706

Epoch 00028: val_loss did not improve from 67.86678
Epoch 29/2000
147/147 [==============================] - 83s 562ms/step - loss: 70.7592 - root_mean_squared_error: 8.4078 -

147/147 [==============================] - 83s 566ms/step - loss: 64.7538 - root_mean_squared_error: 8.0371 - mean_absolute_error: 6.4789 - val_loss: 66.1308 - val_root_mean_squared_error: 8.1321 - val_mean_absolute_error: 6.6338

Epoch 00079: val_loss did not improve from 58.25088
Epoch 80/2000
147/147 [==============================] - 83s 565ms/step - loss: 57.5606 - root_mean_squared_error: 7.5740 - mean_absolute_error: 6.0692 - val_loss: 69.1221 - val_root_mean_squared_error: 8.3140 - val_mean_absolute_error: 6.7726

Epoch 00080: val_loss did not improve from 58.25088
Epoch 81/2000
147/147 [==============================] - 82s 561ms/step - loss: 60.8318 - root_mean_squared_error: 7.7956 - mean_absolute_error: 6.2759 - val_loss: 67.9428 - val_root_mean_squared_error: 8.2427 - val_mean_absolute_error: 6.7986

Epoch 00081: val_loss did not improve from 58.25088
Epoch 82/2000
147/147 [==============================] - 83s 568ms/step - loss: 57.1697 - root_mean_squared_error: 7.5598 -


Epoch 00133: val_loss did not improve from 49.68152
Epoch 134/2000
147/147 [==============================] - 83s 566ms/step - loss: 61.4095 - root_mean_squared_error: 7.8330 - mean_absolute_error: 6.4198 - val_loss: 65.4948 - val_root_mean_squared_error: 8.0929 - val_mean_absolute_error: 6.3631

Epoch 00134: val_loss did not improve from 49.68152
Epoch 135/2000
147/147 [==============================] - 85s 579ms/step - loss: 55.8383 - root_mean_squared_error: 7.4701 - mean_absolute_error: 5.9984 - val_loss: 55.2323 - val_root_mean_squared_error: 7.4318 - val_mean_absolute_error: 6.1769

Epoch 00135: val_loss did not improve from 49.68152
Epoch 136/2000
147/147 [==============================] - 82s 559ms/step - loss: 58.3856 - root_mean_squared_error: 7.6393 - mean_absolute_error: 6.1513 - val_loss: 66.3844 - val_root_mean_squared_error: 8.1477 - val_mean_absolute_error: 6.5365

Epoch 00136: val_loss did not improve from 49.68152
Epoch 137/2000
147/147 [=============================

147/147 [==============================] - 82s 557ms/step - loss: 57.0449 - root_mean_squared_error: 7.5501 - mean_absolute_error: 6.0991 - val_loss: 61.1848 - val_root_mean_squared_error: 7.8221 - val_mean_absolute_error: 6.2046

Epoch 00161: val_loss did not improve from 49.68152
Epoch 162/2000
147/147 [==============================] - 83s 562ms/step - loss: 56.9658 - root_mean_squared_error: 7.5407 - mean_absolute_error: 6.0929 - val_loss: 59.7726 - val_root_mean_squared_error: 7.7313 - val_mean_absolute_error: 6.4853

Epoch 00162: val_loss did not improve from 49.68152
Epoch 163/2000
147/147 [==============================] - 82s 557ms/step - loss: 58.6973 - root_mean_squared_error: 7.6554 - mean_absolute_error: 6.2288 - val_loss: 54.5057 - val_root_mean_squared_error: 7.3828 - val_mean_absolute_error: 5.8288

Epoch 00163: val_loss did not improve from 49.68152
Epoch 164/2000
147/147 [==============================] - 82s 556ms/step - loss: 52.4318 - root_mean_squared_error: 7.238

...

147/147 [==============================] - 73s 499ms/step - loss: 49.8538 - root_mean_squared_error: 7.0559 - mean_absolute_error: 5.6718 - val_loss: 58.9820 - val_root_mean_squared_error: 7.6800 - val_mean_absolute_error: 6.0766

Epoch 00517: val_loss did not improve from 44.28481
Epoch 518/2000
147/147 [==============================] - 75s 511ms/step - loss: 46.9424 - root_mean_squared_error: 6.8509 - mean_absolute_error: 5.6368 - val_loss: 59.2044 - val_root_mean_squared_error: 7.6944 - val_mean_absolute_error: 5.8758

Epoch 00518: val_loss did not improve from 44.28481
Epoch 519/2000
147/147 [==============================] - 73s 495ms/step - loss: 49.8127 - root_mean_squared_error: 7.0556 - mean_absolute_error: 5.8128 - val_loss: 57.3633 - val_root_mean_squared_error: 7.5739 - val_mean_absolute_error: 6.0821

Epoch 00519: val_loss did not improve from 44.28481
Epoch 520/2000
147/147 [==============================] - 73s 498ms/step - loss: 56.4755 - root_mean_squared_error: 7.512


Epoch 00544: val_loss did not improve from 44.28481
Epoch 545/2000
147/147 [==============================] - 1623s 11s/step - loss: 52.4055 - root_mean_squared_error: 7.2353 - mean_absolute_error: 5.7621 - val_loss: 58.3420 - val_root_mean_squared_error: 7.6382 - val_mean_absolute_error: 6.2035

Epoch 00545: val_loss did not improve from 44.28481
Epoch 546/2000
147/147 [==============================] - 291s 2s/step - loss: 49.4606 - root_mean_squared_error: 7.0297 - mean_absolute_error: 5.6819 - val_loss: 53.8930 - val_root_mean_squared_error: 7.3412 - val_mean_absolute_error: 5.7635

Epoch 00546: val_loss did not improve from 44.28481
Epoch 547/2000
147/147 [==============================] - 185s 1s/step - loss: 51.7725 - root_mean_squared_error: 7.1904 - mean_absolute_error: 5.8017 - val_loss: 62.4694 - val_root_mean_squared_error: 7.9038 - val_mean_absolute_error: 6.4220

Epoch 00547: val_loss did not improve from 44.28481

Epoch 00547: ReduceLROnPlateau reducing learning rate to


Epoch 00599: val_loss did not improve from 44.28481
Epoch 600/2000
147/147 [==============================] - 90s 614ms/step - loss: 53.3101 - root_mean_squared_error: 7.2994 - mean_absolute_error: 5.9947 - val_loss: 59.0309 - val_root_mean_squared_error: 7.6832 - val_mean_absolute_error: 6.2175

Epoch 00600: val_loss did not improve from 44.28481
Epoch 601/2000
147/147 [==============================] - 92s 627ms/step - loss: 53.5897 - root_mean_squared_error: 7.3174 - mean_absolute_error: 5.7919 - val_loss: 58.0858 - val_root_mean_squared_error: 7.6214 - val_mean_absolute_error: 6.1594

Epoch 00601: val_loss did not improve from 44.28481
Epoch 602/2000
147/147 [==============================] - 87s 594ms/step - loss: 45.6306 - root_mean_squared_error: 6.7519 - mean_absolute_error: 5.5290 - val_loss: 59.2318 - val_root_mean_squared_error: 7.6962 - val_mean_absolute_error: 5.9484

Epoch 00602: val_loss did not improve from 44.28481
Epoch 603/2000
147/147 [=============================


Epoch 00654: val_loss did not improve from 44.28481
Epoch 655/2000
147/147 [==============================] - 97s 661ms/step - loss: 48.8990 - root_mean_squared_error: 6.9921 - mean_absolute_error: 5.5939 - val_loss: 56.5780 - val_root_mean_squared_error: 7.5218 - val_mean_absolute_error: 6.0461

Epoch 00655: val_loss did not improve from 44.28481
Epoch 656/2000
147/147 [==============================] - 88s 600ms/step - loss: 51.0471 - root_mean_squared_error: 7.1438 - mean_absolute_error: 5.7093 - val_loss: 55.0429 - val_root_mean_squared_error: 7.4191 - val_mean_absolute_error: 5.9571

Epoch 00656: val_loss did not improve from 44.28481
Epoch 657/2000
147/147 [==============================] - 93s 636ms/step - loss: 51.8335 - root_mean_squared_error: 7.1916 - mean_absolute_error: 5.7760 - val_loss: 57.5085 - val_root_mean_squared_error: 7.5834 - val_mean_absolute_error: 5.9608

Epoch 00657: val_loss did not improve from 44.28481
Epoch 658/2000
147/147 [=============================


Epoch 00709: val_loss did not improve from 44.28481
Epoch 710/2000
147/147 [==============================] - 88s 599ms/step - loss: 54.0851 - root_mean_squared_error: 7.3528 - mean_absolute_error: 6.0675 - val_loss: 55.5114 - val_root_mean_squared_error: 7.4506 - val_mean_absolute_error: 5.8974

Epoch 00710: val_loss did not improve from 44.28481
Epoch 711/2000
147/147 [==============================] - 87s 591ms/step - loss: 57.8679 - root_mean_squared_error: 7.6023 - mean_absolute_error: 6.1071 - val_loss: 55.8045 - val_root_mean_squared_error: 7.4702 - val_mean_absolute_error: 5.9782

Epoch 00711: val_loss did not improve from 44.28481
Epoch 712/2000
147/147 [==============================] - 91s 619ms/step - loss: 49.5242 - root_mean_squared_error: 7.0238 - mean_absolute_error: 5.6372 - val_loss: 54.5676 - val_root_mean_squared_error: 7.3870 - val_mean_absolute_error: 6.0076

Epoch 00712: val_loss did not improve from 44.28481
Epoch 713/2000
147/147 [=============================

# 6. BLSTM-LSTM

In [14]:
def blstm_lstm_model():
    """ Returns the BLSTM-LSTM model from Kaushik et al. (2019). """

    # MARK: This model compresses too much in the last phase, check if possible to improve.

    model = keras.Sequential()

    # BLSTM layer
    model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=input_shape))
    model.add(Dropout(.2))
    model.add(BatchNormalization())

    # LSTM layer
    model.add(LSTM(128, return_sequences=True))
    model.add(BatchNormalization())

    # LSTM layer
    model.add(LSTM(64, return_sequences=True))
    model.add(BatchNormalization())

    # Fully connected layer
    model.add(Dense(32))

    model.add(Dense(n_outputs))

    return model 

# def blstm_lstm_model():
#     """ Returns the BLSTM-LSTM model from Kaushik et al. (2019). """

#     # MARK: This model compresses too much in the last phase, check if possible to improve.

#     model = keras.Sequential()

#     # BLSTM layer
#     model.add(Bidirectional(LSTM(256, return_sequences=True), input_shape=input_shape))
#     model.add(Dropout(.2))
#     model.add(BatchNormalization())

#     # LSTM layer
#     model.add(LSTM(128, return_sequences=True))
#     model.add(BatchNormalization())

#     # LSTM layer
#     model.add(LSTM(64, return_sequences=False))
#     model.add(BatchNormalization())

#     # Fully connected layer
#     model.add(Dense(32))

#     model.add(Dense(n_outputs))

#     return model 

In [15]:
model = blstm_lstm_model()

optimizer = Adam(learning_rate=0.01)    
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

# BLSTM_regressor_01: MSE, Adam, N_average=30, 1500 epochs, ES=250, RLR=50, gaussian=0.01
output_filename = 'BLSTM_regressor_FINAL_LOCAL'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=300, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=50, min_lr=0.0001, verbose=1)

In [16]:
%%time

epochs = 1500

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs, 
                    callbacks=[checkpointer, earlystopper, reduce_lr])

Epoch 1/1500
147/147 [==============================] - 327s 2s/step - loss: 365.5937 - root_mean_squared_error: 18.7136 - mean_absolute_error: 15.5972 - val_loss: 138.1879 - val_root_mean_squared_error: 11.7553 - val_mean_absolute_error: 9.8741

Epoch 00001: val_loss improved from inf to 138.18787, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/BLSTM_regressor_FINAL_LOCAL.hdf5
Epoch 2/1500
147/147 [==============================] - 296s 2s/step - loss: 119.7556 - root_mean_squared_error: 10.9393 - mean_absolute_error: 9.3344 - val_loss: 119.1453 - val_root_mean_squared_error: 10.9154 - val_mean_absolute_error: 9.2381

Epoch 00002: val_loss improved from 138.18787 to 119.14527, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/BLSTM_regressor_FINAL_LOCAL.hdf5
Epoch 3/1500
147/147 [==============================] - 327s 2s/step - loss: 111.6655 - root_mean_squared_error: 10.5637 - mean_absolute_error: 8


Epoch 00026: val_loss did not improve from 103.16543
Epoch 27/1500
147/147 [==============================] - 248s 2s/step - loss: 107.4141 - root_mean_squared_error: 10.3612 - mean_absolute_error: 8.6650 - val_loss: 106.3819 - val_root_mean_squared_error: 10.3142 - val_mean_absolute_error: 8.6392

Epoch 00027: val_loss did not improve from 103.16543
Epoch 28/1500
147/147 [==============================] - 254s 2s/step - loss: 101.9313 - root_mean_squared_error: 10.0942 - mean_absolute_error: 8.4173 - val_loss: 104.5299 - val_root_mean_squared_error: 10.2240 - val_mean_absolute_error: 8.2951

Epoch 00028: val_loss did not improve from 103.16543
Epoch 29/1500
147/147 [==============================] - 244s 2s/step - loss: 107.0262 - root_mean_squared_error: 10.3383 - mean_absolute_error: 8.7224 - val_loss: 129.4055 - val_root_mean_squared_error: 11.3757 - val_mean_absolute_error: 9.6241

Epoch 00029: val_loss did not improve from 103.16543
Epoch 30/1500
147/147 [=======================


Epoch 00052: val_loss did not improve from 102.93622
Epoch 53/1500
147/147 [==============================] - 210s 1s/step - loss: 105.2290 - root_mean_squared_error: 10.2489 - mean_absolute_error: 8.5813 - val_loss: 115.9622 - val_root_mean_squared_error: 10.7686 - val_mean_absolute_error: 8.9797

Epoch 00053: val_loss did not improve from 102.93622
Epoch 54/1500
147/147 [==============================] - 269s 2s/step - loss: 106.1755 - root_mean_squared_error: 10.2997 - mean_absolute_error: 8.5415 - val_loss: 115.5391 - val_root_mean_squared_error: 10.7489 - val_mean_absolute_error: 8.7963

Epoch 00054: val_loss did not improve from 102.93622
Epoch 55/1500
147/147 [==============================] - 247s 2s/step - loss: 112.1563 - root_mean_squared_error: 10.5887 - mean_absolute_error: 8.8908 - val_loss: 116.0394 - val_root_mean_squared_error: 10.7722 - val_mean_absolute_error: 9.0872

Epoch 00055: val_loss did not improve from 102.93622
Epoch 56/1500
147/147 [=======================

147/147 [==============================] - 151s 1s/step - loss: 112.7910 - root_mean_squared_error: 10.6165 - mean_absolute_error: 8.9729 - val_loss: 117.8349 - val_root_mean_squared_error: 10.8552 - val_mean_absolute_error: 9.1880

Epoch 00106: val_loss did not improve from 99.42902
Epoch 107/1500
147/147 [==============================] - 151s 1s/step - loss: 109.0916 - root_mean_squared_error: 10.4405 - mean_absolute_error: 8.7619 - val_loss: 113.7383 - val_root_mean_squared_error: 10.6648 - val_mean_absolute_error: 8.9935

Epoch 00107: val_loss did not improve from 99.42902
Epoch 108/1500
147/147 [==============================] - 152s 1s/step - loss: 108.5130 - root_mean_squared_error: 10.4096 - mean_absolute_error: 8.7937 - val_loss: 99.7564 - val_root_mean_squared_error: 9.9878 - val_mean_absolute_error: 8.3393

Epoch 00108: val_loss did not improve from 99.42902
Epoch 109/1500
147/147 [==============================] - 151s 1s/step - loss: 109.3312 - root_mean_squared_error: 10


Epoch 00132: val_loss did not improve from 92.62270
Epoch 133/1500
147/147 [==============================] - 150s 1s/step - loss: 109.0112 - root_mean_squared_error: 10.4361 - mean_absolute_error: 8.8295 - val_loss: 107.2802 - val_root_mean_squared_error: 10.3576 - val_mean_absolute_error: 8.7524

Epoch 00133: val_loss did not improve from 92.62270
Epoch 134/1500
147/147 [==============================] - 153s 1s/step - loss: 105.2973 - root_mean_squared_error: 10.2588 - mean_absolute_error: 8.6503 - val_loss: 109.9992 - val_root_mean_squared_error: 10.4881 - val_mean_absolute_error: 8.5958

Epoch 00134: val_loss did not improve from 92.62270
Epoch 135/1500
147/147 [==============================] - 161s 1s/step - loss: 108.5624 - root_mean_squared_error: 10.4139 - mean_absolute_error: 8.7200 - val_loss: 109.9234 - val_root_mean_squared_error: 10.4844 - val_mean_absolute_error: 8.8504

Epoch 00135: val_loss did not improve from 92.62270
Epoch 136/1500
147/147 [=======================

...


Epoch 00732: val_loss did not improve from 81.02067
Epoch 733/1500
147/147 [==============================] - 170s 1s/step - loss: 96.3230 - root_mean_squared_error: 9.8130 - mean_absolute_error: 8.2368 - val_loss: 92.1015 - val_root_mean_squared_error: 9.5970 - val_mean_absolute_error: 7.8268

Epoch 00733: val_loss did not improve from 81.02067
Epoch 734/1500
147/147 [==============================] - 171s 1s/step - loss: 93.6737 - root_mean_squared_error: 9.6766 - mean_absolute_error: 8.0713 - val_loss: 98.7144 - val_root_mean_squared_error: 9.9355 - val_mean_absolute_error: 8.1775

Epoch 00734: val_loss did not improve from 81.02067
Epoch 735/1500
147/147 [==============================] - 170s 1s/step - loss: 89.2315 - root_mean_squared_error: 9.4430 - mean_absolute_error: 8.0023 - val_loss: 95.9576 - val_root_mean_squared_error: 9.7958 - val_mean_absolute_error: 8.2594

Epoch 00735: val_loss did not improve from 81.02067
Epoch 736/1500
147/147 [==============================] - 1


Epoch 00787: val_loss did not improve from 81.02067
Epoch 788/1500
147/147 [==============================] - 282s 2s/step - loss: 99.1110 - root_mean_squared_error: 9.9518 - mean_absolute_error: 8.5053 - val_loss: 99.3724 - val_root_mean_squared_error: 9.9686 - val_mean_absolute_error: 8.3297

Epoch 00788: val_loss did not improve from 81.02067
Epoch 789/1500
147/147 [==============================] - 283s 2s/step - loss: 88.3363 - root_mean_squared_error: 9.3951 - mean_absolute_error: 7.9224 - val_loss: 92.8673 - val_root_mean_squared_error: 9.6368 - val_mean_absolute_error: 8.1059

Epoch 00789: val_loss did not improve from 81.02067
Epoch 790/1500
147/147 [==============================] - 312s 2s/step - loss: 99.2065 - root_mean_squared_error: 9.9584 - mean_absolute_error: 8.3269 - val_loss: 94.8875 - val_root_mean_squared_error: 9.7410 - val_mean_absolute_error: 8.0228

Epoch 00790: val_loss did not improve from 81.02067
Epoch 791/1500
147/147 [==============================] - 2


Epoch 00842: val_loss did not improve from 81.02067
Epoch 843/1500
147/147 [==============================] - 245s 2s/step - loss: 84.1353 - root_mean_squared_error: 9.1685 - mean_absolute_error: 7.6655 - val_loss: 95.5905 - val_root_mean_squared_error: 9.7770 - val_mean_absolute_error: 8.2040

Epoch 00843: val_loss did not improve from 81.02067
Epoch 844/1500
147/147 [==============================] - 259s 2s/step - loss: 90.9115 - root_mean_squared_error: 9.5329 - mean_absolute_error: 7.9749 - val_loss: 96.5300 - val_root_mean_squared_error: 9.8250 - val_mean_absolute_error: 8.2474

Epoch 00844: val_loss did not improve from 81.02067
Epoch 845/1500
147/147 [==============================] - 261s 2s/step - loss: 90.4489 - root_mean_squared_error: 9.5094 - mean_absolute_error: 8.0455 - val_loss: 97.1553 - val_root_mean_squared_error: 9.8567 - val_mean_absolute_error: 8.3414

Epoch 00845: val_loss did not improve from 81.02067
Epoch 846/1500
147/147 [==============================] - 2


Epoch 00897: val_loss did not improve from 81.02067
Epoch 898/1500
147/147 [==============================] - 184s 1s/step - loss: 89.0897 - root_mean_squared_error: 9.4374 - mean_absolute_error: 7.8500 - val_loss: 88.9504 - val_root_mean_squared_error: 9.4313 - val_mean_absolute_error: 7.7807

Epoch 00898: val_loss did not improve from 81.02067
Epoch 899/1500
147/147 [==============================] - 184s 1s/step - loss: 89.9124 - root_mean_squared_error: 9.4816 - mean_absolute_error: 7.9683 - val_loss: 91.0076 - val_root_mean_squared_error: 9.5398 - val_mean_absolute_error: 7.9797

Epoch 00899: val_loss did not improve from 81.02067
Epoch 900/1500
147/147 [==============================] - 185s 1s/step - loss: 92.9195 - root_mean_squared_error: 9.6372 - mean_absolute_error: 8.0440 - val_loss: 96.1819 - val_root_mean_squared_error: 9.8072 - val_mean_absolute_error: 8.0649

Epoch 00900: val_loss did not improve from 81.02067
Epoch 901/1500
147/147 [==============================] - 1


Epoch 00952: val_loss did not improve from 81.02067
Epoch 953/1500
147/147 [==============================] - 178s 1s/step - loss: 90.4738 - root_mean_squared_error: 9.5086 - mean_absolute_error: 8.0123 - val_loss: 97.5860 - val_root_mean_squared_error: 9.8786 - val_mean_absolute_error: 8.1039

Epoch 00953: val_loss did not improve from 81.02067
Epoch 00953: early stopping
CPU times: user 9d 6h 58min 3s, sys: 1d 19h 7min 33s, total: 11d 2h 5min 36s
Wall time: 2d 10h 16min 35s


# 7. InceptionTime

In [17]:
class Regressor_Inception:

    def __init__(self, output_directory, input_shape, verbose=False, build=True, batch_size=64,
                 nb_filters=32, use_residual=True, use_bottleneck=True, depth=6, kernel_size=41, nb_epochs=1500):

        self.output_directory = output_directory

        self.nb_filters = nb_filters
        self.use_residual = use_residual
        self.use_bottleneck = use_bottleneck
        self.depth = depth
        self.kernel_size = kernel_size - 1
        self.callbacks = None
        self.batch_size = batch_size
        self.bottleneck_size = 32
        self.nb_epochs = nb_epochs

        if build == True:
            self.model = self.build_model(input_shape)
            if (verbose == True):
                self.model.summary()
            self.verbose = verbose
            self.model.save_weights(self.output_directory + 'inception_model_init.hdf5')

    def _inception_module(self, input_tensor, stride=1, activation='linear'):

        if self.use_bottleneck and int(input_tensor.shape[-1]) > 1:
            input_inception = tf.keras.layers.Conv1D(filters=self.bottleneck_size, kernel_size=1,
                                                  padding='same', activation=activation, use_bias=False)(input_tensor)
        else:
            input_inception = input_tensor

        # kernel_size_s = [3, 5, 8, 11, 17]
        kernel_size_s = [self.kernel_size // (2 ** i) for i in range(3)]

        conv_list = []

        for i in range(len(kernel_size_s)):
            conv_list.append(tf.keras.layers.Conv1D(filters=self.nb_filters, kernel_size=kernel_size_s[i],
                                                 strides=stride, padding='same', activation=activation, use_bias=False)(
                input_inception))

        max_pool_1 = tf.keras.layers.MaxPool1D(pool_size=3, strides=stride, padding='same')(input_tensor)

        conv_6 = tf.keras.layers.Conv1D(filters=self.nb_filters, kernel_size=1,
                                     padding='same', activation=activation, use_bias=False)(max_pool_1)

        conv_list.append(conv_6)

        x = tf.keras.layers.Concatenate(axis=2)(conv_list)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation(activation='relu')(x)
        return x

    def _shortcut_layer(self, input_tensor, out_tensor):
        shortcut_y = tf.keras.layers.Conv1D(filters=int(out_tensor.shape[-1]), kernel_size=1,
                                         padding='same', use_bias=False)(input_tensor)
        shortcut_y = tf.keras.layers.BatchNormalization()(shortcut_y)

        x = tf.keras.layers.Add()([shortcut_y, out_tensor])
        x = tf.keras.layers.Activation('relu')(x)
        return x

    def build_model(self, input_shape):
        input_layer = tf.keras.layers.Input(input_shape)

        x = input_layer
        input_res = input_layer

        for d in range(self.depth):

            x = self._inception_module(x)

            if self.use_residual and d % 3 == 2:
                x = self._shortcut_layer(input_res, x)
                input_res = x

        pooling_layer = tf.keras.layers.AveragePooling1D(pool_size=50)(x)
        flat_layer = tf.keras.layers.Flatten()(pooling_layer)
        dense_layer = tf.keras.layers.Dense(128, activation='relu')(flat_layer)
        output_layer = tf.keras.layers.Dense(1)(dense_layer)

        model = tf.keras.models.Model(inputs=input_layer, outputs=output_layer)

        return model

In [18]:
model = Regressor_Inception(PATH_MODELS, input_shape, verbose=True).model

optimizer = Adam(learning_rate=0.01)   
              
model.compile(loss='mean_squared_error', 
              optimizer=optimizer, 
              metrics=[RootMeanSquaredError(), MeanAbsoluteError()])

# 'Inception_regressor_01' (n_average = 40, gaussian_noise = 0.01, MAE)
# 'Inception_regressor_02' (n_average = 1, gaussian_noise = 0.01, MAE)
# 'Inception_regressor_03' (n_average = 40, gaussian_noise = 0.01, MSE)
# 'Inception_regressor_04' (n_average = 1, gaussian_noise = 0.01, MSE)
# 'Inception_regressor_05' (n_average = 100, gaussian_noise = 0.01, MAE)
output_filename = 'Inception_regressor_FINAL_LOCAL'
output_file = os.path.join(PATH_MODELS, output_filename)

checkpointer = ModelCheckpoint(filepath = output_file + ".hdf5", monitor='val_loss', verbose=1, save_best_only=True)
earlystopper = EarlyStopping(monitor='val_loss', patience=180, verbose=1)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=30, min_lr=0.0001, verbose=1)

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 501, 30)]    0                                            
__________________________________________________________________________________________________
conv1d_2 (Conv1D)               (None, 501, 32)      960         input_2[0][0]                    
__________________________________________________________________________________________________
max_pooling1d (MaxPooling1D)    (None, 501, 30)      0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1d_3 (Conv1D)               (None, 501, 32)      40960       conv1d_2[0][0]                   
____________________________________________________________________________________________

In [19]:
%%time

#REMOVE THIS TEMP CELL
epochs = 1500

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs,
                    callbacks = [checkpointer, earlystopper, reduce_lr])

Epoch 1/1500
147/147 [==============================] - 123s 819ms/step - loss: 182.6402 - root_mean_squared_error: 13.1730 - mean_absolute_error: 10.5014 - val_loss: 114.5923 - val_root_mean_squared_error: 10.7048 - val_mean_absolute_error: 8.7120

Epoch 00001: val_loss improved from inf to 114.59228, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Inception_regressor_FINAL_LOCAL.hdf5
Epoch 2/1500
147/147 [==============================] - 117s 796ms/step - loss: 97.9022 - root_mean_squared_error: 9.8918 - mean_absolute_error: 8.1666 - val_loss: 104.8572 - val_root_mean_squared_error: 10.2400 - val_mean_absolute_error: 8.1554

Epoch 00002: val_loss improved from 114.59228 to 104.85725, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Inception_regressor_FINAL_LOCAL.hdf5
Epoch 3/1500
147/147 [==============================] - 117s 792ms/step - loss: 98.3949 - root_mean_squared_error: 9.9162 - mean_abso

147/147 [==============================] - 115s 784ms/step - loss: 61.9315 - root_mean_squared_error: 7.8684 - mean_absolute_error: 6.2474 - val_loss: 51.6124 - val_root_mean_squared_error: 7.1842 - val_mean_absolute_error: 5.6374

Epoch 00025: val_loss improved from 57.48402 to 51.61241, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Inception_regressor_FINAL_LOCAL.hdf5
Epoch 26/1500
147/147 [==============================] - 117s 796ms/step - loss: 64.9962 - root_mean_squared_error: 8.0599 - mean_absolute_error: 6.5079 - val_loss: 65.2700 - val_root_mean_squared_error: 8.0790 - val_mean_absolute_error: 6.5160

Epoch 00026: val_loss did not improve from 51.61241
Epoch 27/1500
147/147 [==============================] - 115s 785ms/step - loss: 63.0132 - root_mean_squared_error: 7.9329 - mean_absolute_error: 6.4490 - val_loss: 63.6171 - val_root_mean_squared_error: 7.9760 - val_mean_absolute_error: 6.0435

Epoch 00027: val_loss did not improve 


Epoch 00051: val_loss did not improve from 49.10678
Epoch 52/1500
147/147 [==============================] - 118s 803ms/step - loss: 53.3374 - root_mean_squared_error: 7.3008 - mean_absolute_error: 5.8127 - val_loss: 59.9207 - val_root_mean_squared_error: 7.7408 - val_mean_absolute_error: 5.9672

Epoch 00052: val_loss did not improve from 49.10678
Epoch 53/1500
147/147 [==============================] - 119s 810ms/step - loss: 55.8891 - root_mean_squared_error: 7.4700 - mean_absolute_error: 5.8374 - val_loss: 64.4026 - val_root_mean_squared_error: 8.0251 - val_mean_absolute_error: 6.1251

Epoch 00053: val_loss did not improve from 49.10678
Epoch 54/1500
147/147 [==============================] - 119s 813ms/step - loss: 55.8968 - root_mean_squared_error: 7.4686 - mean_absolute_error: 5.9950 - val_loss: 58.4253 - val_root_mean_squared_error: 7.6436 - val_mean_absolute_error: 5.9747

Epoch 00054: val_loss did not improve from 49.10678
Epoch 55/1500
147/147 [==============================


Epoch 00078: val_loss did not improve from 44.23303
Epoch 79/1500
147/147 [==============================] - 118s 800ms/step - loss: 50.6048 - root_mean_squared_error: 7.1072 - mean_absolute_error: 5.7324 - val_loss: 61.6431 - val_root_mean_squared_error: 7.8513 - val_mean_absolute_error: 6.3622

Epoch 00079: val_loss did not improve from 44.23303
Epoch 80/1500
147/147 [==============================] - 116s 789ms/step - loss: 51.6296 - root_mean_squared_error: 7.1819 - mean_absolute_error: 5.7324 - val_loss: 56.8549 - val_root_mean_squared_error: 7.5402 - val_mean_absolute_error: 6.0078

Epoch 00080: val_loss did not improve from 44.23303
Epoch 81/1500
147/147 [==============================] - 117s 795ms/step - loss: 46.8169 - root_mean_squared_error: 6.8315 - mean_absolute_error: 5.4909 - val_loss: 52.6815 - val_root_mean_squared_error: 7.2582 - val_mean_absolute_error: 5.5630

Epoch 00081: val_loss did not improve from 44.23303
Epoch 82/1500
147/147 [==============================

147/147 [==============================] - 116s 792ms/step - loss: 46.8270 - root_mean_squared_error: 6.8405 - mean_absolute_error: 5.4871 - val_loss: 58.0959 - val_root_mean_squared_error: 7.6221 - val_mean_absolute_error: 5.7715

Epoch 00106: val_loss did not improve from 38.98880
Epoch 107/1500
147/147 [==============================] - 119s 810ms/step - loss: 42.4414 - root_mean_squared_error: 6.5135 - mean_absolute_error: 5.1991 - val_loss: 42.4506 - val_root_mean_squared_error: 6.5154 - val_mean_absolute_error: 5.1480

Epoch 00107: val_loss did not improve from 38.98880
Epoch 108/1500
147/147 [==============================] - 118s 805ms/step - loss: 43.2641 - root_mean_squared_error: 6.5710 - mean_absolute_error: 5.1836 - val_loss: 46.3463 - val_root_mean_squared_error: 6.8078 - val_mean_absolute_error: 5.4236

Epoch 00108: val_loss did not improve from 38.98880
Epoch 109/1500
147/147 [==============================] - 119s 807ms/step - loss: 42.5973 - root_mean_squared_error: 6

147/147 [==============================] - 119s 811ms/step - loss: 44.9722 - root_mean_squared_error: 6.7035 - mean_absolute_error: 5.3609 - val_loss: 49.9291 - val_root_mean_squared_error: 7.0661 - val_mean_absolute_error: 5.3979

Epoch 00133: val_loss did not improve from 37.83955
Epoch 134/1500
147/147 [==============================] - 119s 808ms/step - loss: 41.5700 - root_mean_squared_error: 6.4403 - mean_absolute_error: 5.0558 - val_loss: 57.7221 - val_root_mean_squared_error: 7.5975 - val_mean_absolute_error: 5.9150

Epoch 00134: val_loss did not improve from 37.83955
Epoch 135/1500
147/147 [==============================] - 118s 803ms/step - loss: 39.3619 - root_mean_squared_error: 6.2722 - mean_absolute_error: 5.0435 - val_loss: 44.1624 - val_root_mean_squared_error: 6.6455 - val_mean_absolute_error: 5.1572

Epoch 00135: val_loss did not improve from 37.83955
Epoch 136/1500
147/147 [==============================] - 117s 793ms/step - loss: 43.0164 - root_mean_squared_error: 6

147/147 [==============================] - 119s 813ms/step - loss: 35.4372 - root_mean_squared_error: 5.9506 - mean_absolute_error: 4.6522 - val_loss: 54.8123 - val_root_mean_squared_error: 7.4035 - val_mean_absolute_error: 5.5608

Epoch 00187: val_loss did not improve from 37.83955
Epoch 188/1500
147/147 [==============================] - 119s 807ms/step - loss: 34.8226 - root_mean_squared_error: 5.8937 - mean_absolute_error: 4.6231 - val_loss: 59.5987 - val_root_mean_squared_error: 7.7200 - val_mean_absolute_error: 5.8403

Epoch 00188: val_loss did not improve from 37.83955

Epoch 00188: ReduceLROnPlateau reducing learning rate to 0.0006249999860301614.
Epoch 189/1500
147/147 [==============================] - 117s 795ms/step - loss: 40.0559 - root_mean_squared_error: 6.3266 - mean_absolute_error: 4.9938 - val_loss: 56.8687 - val_root_mean_squared_error: 7.5411 - val_mean_absolute_error: 5.4989

Epoch 00189: val_loss did not improve from 37.83955
Epoch 190/1500
147/147 [=============

147/147 [==============================] - 119s 808ms/step - loss: 43.3186 - root_mean_squared_error: 6.5784 - mean_absolute_error: 5.1857 - val_loss: 48.7543 - val_root_mean_squared_error: 6.9824 - val_mean_absolute_error: 5.4665

Epoch 00241: val_loss did not improve from 37.83955
Epoch 242/1500
147/147 [==============================] - 118s 804ms/step - loss: 41.1628 - root_mean_squared_error: 6.4145 - mean_absolute_error: 5.0942 - val_loss: 54.9014 - val_root_mean_squared_error: 7.4095 - val_mean_absolute_error: 5.3745

Epoch 00242: val_loss did not improve from 37.83955
Epoch 243/1500
147/147 [==============================] - 119s 809ms/step - loss: 39.5870 - root_mean_squared_error: 6.2903 - mean_absolute_error: 4.9175 - val_loss: 44.5408 - val_root_mean_squared_error: 6.6739 - val_mean_absolute_error: 5.0988

Epoch 00243: val_loss did not improve from 37.83955
Epoch 244/1500
147/147 [==============================] - 117s 799ms/step - loss: 38.6962 - root_mean_squared_error: 6

147/147 [==============================] - 118s 801ms/step - loss: 35.0401 - root_mean_squared_error: 5.9170 - mean_absolute_error: 4.6795 - val_loss: 50.2917 - val_root_mean_squared_error: 7.0917 - val_mean_absolute_error: 5.4698

Epoch 00295: val_loss did not improve from 37.83955
Epoch 296/1500
147/147 [==============================] - 113s 770ms/step - loss: 38.9478 - root_mean_squared_error: 6.2393 - mean_absolute_error: 4.8730 - val_loss: 54.6647 - val_root_mean_squared_error: 7.3936 - val_mean_absolute_error: 5.5642

Epoch 00296: val_loss did not improve from 37.83955
Epoch 297/1500
147/147 [==============================] - 114s 779ms/step - loss: 38.6799 - root_mean_squared_error: 6.2129 - mean_absolute_error: 4.8933 - val_loss: 47.3670 - val_root_mean_squared_error: 6.8824 - val_mean_absolute_error: 5.3857

Epoch 00297: val_loss did not improve from 37.83955
Epoch 298/1500
147/147 [==============================] - 113s 770ms/step - loss: 35.7264 - root_mean_squared_error: 5

In [96]:
%%time

epochs = 1500

# fit network
history = model.fit(x=train_generator_noise,
                    validation_data=val_generator,
                    epochs=epochs,
                    callbacks = [checkpointer, earlystopper, reduce_lr])

Epoch 1/1500
143/143 [==============================] - 249s 2s/step - loss: 14.8795 - root_mean_squared_error: 21.7512 - mean_absolute_error: 14.8795 - val_loss: 14.4589 - val_root_mean_squared_error: 18.1789 - val_mean_absolute_error: 14.4589

Epoch 00001: val_loss improved from inf to 14.45891, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Inception_regressor_05.hdf5
Epoch 2/1500
143/143 [==============================] - 238s 2s/step - loss: 8.9878 - root_mean_squared_error: 11.2585 - mean_absolute_error: 8.9878 - val_loss: 8.4816 - val_root_mean_squared_error: 10.7197 - val_mean_absolute_error: 8.4816

Epoch 00002: val_loss improved from 14.45891 to 8.48164, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Inception_regressor_05.hdf5
Epoch 3/1500
143/143 [==============================] - 241s 2s/step - loss: 7.9297 - root_mean_squared_error: 9.8073 - mean_absolute_error: 7.9297 - val_loss: 8.62


Epoch 00024: val_loss did not improve from 5.82968
Epoch 25/1500
143/143 [==============================] - 249s 2s/step - loss: 5.9342 - root_mean_squared_error: 7.4770 - mean_absolute_error: 5.9342 - val_loss: 5.8190 - val_root_mean_squared_error: 7.3576 - val_mean_absolute_error: 5.8190

Epoch 00025: val_loss improved from 5.82968 to 5.81904, saving model to /Users/bbruns/Desktop/Studie/Thesis/Code/EEG_age_prediction/trained_models/Inception_regressor_05.hdf5
Epoch 26/1500
143/143 [==============================] - 244s 2s/step - loss: 5.8491 - root_mean_squared_error: 7.5007 - mean_absolute_error: 5.8491 - val_loss: 6.2931 - val_root_mean_squared_error: 7.9237 - val_mean_absolute_error: 6.2931

Epoch 00026: val_loss did not improve from 5.81904
Epoch 27/1500
143/143 [==============================] - 234s 2s/step - loss: 5.7424 - root_mean_squared_error: 7.2581 - mean_absolute_error: 5.7424 - val_loss: 5.7668 - val_root_mean_squared_error: 7.2636 - val_mean_absolute_error: 5.7668


143/143 [==============================] - 316s 2s/step - loss: 5.1042 - root_mean_squared_error: 6.6133 - mean_absolute_error: 5.1042 - val_loss: 5.6670 - val_root_mean_squared_error: 7.2707 - val_mean_absolute_error: 5.6670

Epoch 00078: val_loss did not improve from 5.23083
Epoch 79/1500
143/143 [==============================] - 321s 2s/step - loss: 4.9956 - root_mean_squared_error: 6.3485 - mean_absolute_error: 4.9956 - val_loss: 5.4951 - val_root_mean_squared_error: 7.2871 - val_mean_absolute_error: 5.4951

Epoch 00079: val_loss did not improve from 5.23083
Epoch 80/1500
143/143 [==============================] - 324s 2s/step - loss: 5.1040 - root_mean_squared_error: 6.4311 - mean_absolute_error: 5.1040 - val_loss: 5.6553 - val_root_mean_squared_error: 7.0254 - val_mean_absolute_error: 5.6553

Epoch 00080: val_loss did not improve from 5.23083
Epoch 81/1500
143/143 [==============================] - 324s 2s/step - loss: 4.8514 - root_mean_squared_error: 6.2461 - mean_absolute_err

143/143 [==============================] - 303s 2s/step - loss: 4.9634 - root_mean_squared_error: 6.4851 - mean_absolute_error: 4.9634 - val_loss: 6.0805 - val_root_mean_squared_error: 7.7360 - val_mean_absolute_error: 6.0805

Epoch 00106: val_loss did not improve from 5.23083
Epoch 107/1500
143/143 [==============================] - 323s 2s/step - loss: 4.9089 - root_mean_squared_error: 6.3690 - mean_absolute_error: 4.9089 - val_loss: 5.7419 - val_root_mean_squared_error: 7.3350 - val_mean_absolute_error: 5.7419

Epoch 00107: val_loss did not improve from 5.23083
Epoch 108/1500
143/143 [==============================] - 324s 2s/step - loss: 4.6513 - root_mean_squared_error: 5.9217 - mean_absolute_error: 4.6513 - val_loss: 6.1858 - val_root_mean_squared_error: 7.8471 - val_mean_absolute_error: 6.1858

Epoch 00108: val_loss did not improve from 5.23083
Epoch 109/1500
143/143 [==============================] - 321s 2s/step - loss: 5.0534 - root_mean_squared_error: 6.3555 - mean_absolute_


Epoch 00133: val_loss did not improve from 5.20694
Epoch 134/1500
143/143 [==============================] - 323s 2s/step - loss: 4.9270 - root_mean_squared_error: 6.2901 - mean_absolute_error: 4.9270 - val_loss: 5.5962 - val_root_mean_squared_error: 7.2905 - val_mean_absolute_error: 5.5962

Epoch 00134: val_loss did not improve from 5.20694
Epoch 135/1500
143/143 [==============================] - 323s 2s/step - loss: 4.8693 - root_mean_squared_error: 6.1092 - mean_absolute_error: 4.8693 - val_loss: 5.6170 - val_root_mean_squared_error: 7.3019 - val_mean_absolute_error: 5.6170

Epoch 00135: val_loss did not improve from 5.20694
Epoch 136/1500
143/143 [==============================] - 323s 2s/step - loss: 4.7565 - root_mean_squared_error: 6.1799 - mean_absolute_error: 4.7565 - val_loss: 5.7584 - val_root_mean_squared_error: 7.4199 - val_mean_absolute_error: 5.7584

Epoch 00136: val_loss did not improve from 5.20694
Epoch 137/1500
143/143 [==============================] - 323s 2s/ste

143/143 [==============================] - 323s 2s/step - loss: 4.6652 - root_mean_squared_error: 6.0196 - mean_absolute_error: 4.6652 - val_loss: 6.1782 - val_root_mean_squared_error: 7.6756 - val_mean_absolute_error: 6.1782

Epoch 00189: val_loss did not improve from 5.04051
Epoch 190/1500
143/143 [==============================] - 323s 2s/step - loss: 4.8163 - root_mean_squared_error: 6.2274 - mean_absolute_error: 4.8163 - val_loss: 5.3501 - val_root_mean_squared_error: 6.8194 - val_mean_absolute_error: 5.3501

Epoch 00190: val_loss did not improve from 5.04051
Epoch 191/1500
143/143 [==============================] - 323s 2s/step - loss: 4.8694 - root_mean_squared_error: 6.2544 - mean_absolute_error: 4.8694 - val_loss: 5.5560 - val_root_mean_squared_error: 7.1667 - val_mean_absolute_error: 5.5560

Epoch 00191: val_loss did not improve from 5.04051
Epoch 192/1500
143/143 [==============================] - 324s 2s/step - loss: 4.8435 - root_mean_squared_error: 6.1235 - mean_absolute_


Epoch 00216: val_loss did not improve from 4.69455
Epoch 217/1500
143/143 [==============================] - 293s 2s/step - loss: 4.8800 - root_mean_squared_error: 6.2364 - mean_absolute_error: 4.8800 - val_loss: 5.8352 - val_root_mean_squared_error: 7.3876 - val_mean_absolute_error: 5.8352

Epoch 00217: val_loss did not improve from 4.69455
Epoch 218/1500
143/143 [==============================] - 284s 2s/step - loss: 4.6264 - root_mean_squared_error: 5.8839 - mean_absolute_error: 4.6264 - val_loss: 5.5292 - val_root_mean_squared_error: 7.0442 - val_mean_absolute_error: 5.5292

Epoch 00218: val_loss did not improve from 4.69455
Epoch 219/1500
143/143 [==============================] - 311s 2s/step - loss: 5.0450 - root_mean_squared_error: 6.4782 - mean_absolute_error: 5.0450 - val_loss: 5.7673 - val_root_mean_squared_error: 7.2827 - val_mean_absolute_error: 5.7673

Epoch 00219: val_loss did not improve from 4.69455
Epoch 220/1500
143/143 [==============================] - 276s 2s/ste


Epoch 00244: val_loss did not improve from 4.69455
Epoch 245/1500
143/143 [==============================] - 1083s 8s/step - loss: 4.8373 - root_mean_squared_error: 6.3008 - mean_absolute_error: 4.8373 - val_loss: 5.7162 - val_root_mean_squared_error: 7.2399 - val_mean_absolute_error: 5.7162

Epoch 00245: val_loss did not improve from 4.69455
Epoch 246/1500
143/143 [==============================] - 854s 6s/step - loss: 4.7068 - root_mean_squared_error: 6.0775 - mean_absolute_error: 4.7068 - val_loss: 5.7267 - val_root_mean_squared_error: 7.3911 - val_mean_absolute_error: 5.7267

Epoch 00246: val_loss did not improve from 4.69455
Epoch 247/1500
143/143 [==============================] - 484s 3s/step - loss: 4.9483 - root_mean_squared_error: 6.2214 - mean_absolute_error: 4.9483 - val_loss: 5.1563 - val_root_mean_squared_error: 6.7706 - val_mean_absolute_error: 5.1563

Epoch 00247: val_loss did not improve from 4.69455
Epoch 248/1500
143/143 [==============================] - 319s 2s/st


Epoch 00272: val_loss did not improve from 4.69455
Epoch 273/1500
143/143 [==============================] - 289s 2s/step - loss: 4.9148 - root_mean_squared_error: 6.3654 - mean_absolute_error: 4.9148 - val_loss: 5.6706 - val_root_mean_squared_error: 7.2174 - val_mean_absolute_error: 5.6706

Epoch 00273: val_loss did not improve from 4.69455
Epoch 274/1500
143/143 [==============================] - 276s 2s/step - loss: 4.6158 - root_mean_squared_error: 6.1187 - mean_absolute_error: 4.6158 - val_loss: 5.6966 - val_root_mean_squared_error: 7.5244 - val_mean_absolute_error: 5.6966

Epoch 00274: val_loss did not improve from 4.69455
Epoch 275/1500
143/143 [==============================] - 298s 2s/step - loss: 5.0629 - root_mean_squared_error: 6.5235 - mean_absolute_error: 5.0629 - val_loss: 6.1959 - val_root_mean_squared_error: 7.6484 - val_mean_absolute_error: 6.1959

Epoch 00275: val_loss did not improve from 4.69455
Epoch 276/1500
143/143 [==============================] - 287s 2s/ste


Epoch 00300: val_loss did not improve from 4.69455
Epoch 301/1500
143/143 [==============================] - 321s 2s/step - loss: 4.8422 - root_mean_squared_error: 6.2162 - mean_absolute_error: 4.8422 - val_loss: 5.7874 - val_root_mean_squared_error: 7.3298 - val_mean_absolute_error: 5.7874

Epoch 00301: val_loss did not improve from 4.69455
Epoch 302/1500
143/143 [==============================] - 323s 2s/step - loss: 4.9055 - root_mean_squared_error: 6.4589 - mean_absolute_error: 4.9055 - val_loss: 5.7145 - val_root_mean_squared_error: 7.0399 - val_mean_absolute_error: 5.7145

Epoch 00302: val_loss did not improve from 4.69455
Epoch 303/1500
143/143 [==============================] - 322s 2s/step - loss: 4.8142 - root_mean_squared_error: 6.1713 - mean_absolute_error: 4.8142 - val_loss: 5.3505 - val_root_mean_squared_error: 6.8226 - val_mean_absolute_error: 5.3505

Epoch 00303: val_loss did not improve from 4.69455
Epoch 304/1500
143/143 [==============================] - 300s 2s/ste

# Compare models

### Helper functions for comparison

In [92]:
# 'Inception_regressor_01' (n_average = 40, gaussian_noise = 0.01, MAE)
model_path = os.path.join(PATH_MODELS, 'Inception_regressor_01.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 17s 474ms/step - loss: 6.1535 - root_mean_squared_error: 7.8651 - mean_absolute_error: 6.1535


In [93]:
# 'Inception_regressor_02' (n_average = 1, gaussian_noise = 0.01, MAE)
model_path = os.path.join(PATH_MODELS, 'Inception_regressor_02.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 16s 466ms/step - loss: 14.0345 - root_mean_squared_error: 17.0194 - mean_absolute_error: 14.0345


In [94]:
# 'Inception_regressor_03' (n_average = 40, gaussian_noise = 0.01, MSE)
model_path = os.path.join(PATH_MODELS, 'Inception_regressor_03.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 16s 445ms/step - loss: 58.4772 - root_mean_squared_error: 7.6470 - mean_absolute_error: 5.9643


In [95]:
# 'Inception_regressor_04' (n_average = 1, gaussian_noise = 0.01, MSE)
model_path = os.path.join(PATH_MODELS, 'Inception_regressor_04.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 15s 433ms/step - loss: 221.2316 - root_mean_squared_error: 14.8739 - mean_absolute_error: 12.0842


In [96]:
# 'Inception_regressor_05' (n_average = 100, gaussian_noise = 0.01, MAE)
model_path = os.path.join(PATH_MODELS, 'Inception_regressor_05.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 16s 448ms/step - loss: 6.9294 - root_mean_squared_error: 8.5678 - mean_absolute_error: 6.9294


In [58]:
# Fully_connected_regressor_02: MSE, Adadelta, N_average=30, 5000 epochs, ES=1000, RLR=200, gaussian=0.01
model_path = os.path.join(PATH_MODELS, 'Fully_connected_regressor_02.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 12s 342ms/step - loss: 71.6239 - root_mean_squared_error: 8.4631 - mean_absolute_error: 6.5200


In [98]:
# CNN_regressor_01: MSE, Adam, N_average=30, 2000 epochs, ES=250, RLR=50, gaussian=0.01
model_path = os.path.join(PATH_MODELS, 'CNN_regressor_01.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 15s 420ms/step - loss: 62.4980 - root_mean_squared_error: 7.9056 - mean_absolute_error: 6.1974


In [99]:
# ResNet_regressor_01: MSE, Adam, N_average=30, 1500 epochs, ES=250, RLR=50, gaussian=0.01
model_path = os.path.join(PATH_MODELS, 'ResNet_regressor_01.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 17s 473ms/step - loss: 77.0855 - root_mean_squared_error: 8.7798 - mean_absolute_error: 6.6343


In [100]:
# Encoder_regressor_01: MSE, Adam, N_average=30, 1500 epochs, ES=250, RLR=50, gaussian=0.01 (LR = 0.0001, no reduction)
model_path = os.path.join(PATH_MODELS, 'Encoder_regressor_01.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 15s 434ms/step - loss: 73.6694 - root_mean_squared_error: 8.5831 - mean_absolute_error: 6.6446


In [101]:
# TimeCNN_regressor_01: MSE, Adam, N_average=30, 2000 epochs, ES=250, RLR=50, gaussian=0.01
model_path = os.path.join(PATH_MODELS, 'TimeCNN_regressor_01.hdf5')

loaded_model = tf.keras.models.load_model(model_path)
evaluate_model(loaded_model)

35/35 [==============================] - 16s 473ms/step - loss: 62.2474 - root_mean_squared_error: 7.8897 - mean_absolute_error: 6.1511
